## XML предобработка

In [2]:
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss, precision_score, recall_score, f1_score, classification_report, accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from xgboost import plot_importance,XGBClassifier,XGBRegressor
from catboost import CatBoostClassifier
import time
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import multilabel_confusion_matrix
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
from bs4 import BeautifulSoup
import re

xml_content = """
<root>
    <p>ПЕРЕД БЛОКОМ.</p>
    <P ID="P0006" CLASS="formattext topleveltext" ALIGN="justify">
        <SPAN class="comment">12121212</SPAN>
    </P>
    <CHAPMARK ID="P0005" STRING="HIDDEN_BLOCK_OPEN#Информация об изменяющих документах" MARKER="00000000000000000000000000000000000000000000000003U4E8OT" HIDDEN="1" LEVEL="8"/>
    <P ID="P0007" CLASS="formattext topleveltext" ALIGN="justify">
        <SPAN class="comment">
            Документ с изменениями, внесенными:<BR/>
            <EXTERNAL ND="499012712" CONTEXT="0000000000000000000000000000000000000000000000000064U0IK">Федеральным законом от 5 августа 2000 года N 118-ФЗ</EXTERNAL> (изменения вступили в силу с 1 января 2002 года);<BR/>
            <EXTERNAL ND="901777906" CONTEXT="0000000000000000000000000000000000000000000000000064U0IK">Федеральным законом от 29 декабря 2000 года N 166-ФЗ</EXTERNAL> (Российская газета, N 248, 31.12.2000) (вступил в силу с 1 января 2001 года);<BR/>
            <EXTERNAL ND="901788506">Федеральным законом от 30 мая 2001 года N 71-ФЗ</EXTERNAL> (Российская газета, N 105, 02.06.2001);<BR/>
            <EXTERNAL ND="901794410" CONTEXT="0000000000000000000000000000000000000000000000000064U0IK">Федеральным законом от 7 августа 2001 года N 118-ФЗ</EXTERNAL>
            <EXTERNAL ND="901794410" CONTEXT="0000000000000000000000000000000000000000000000000064U0IK">Федеральным законом от 7 августа 2001 года N 118-ФЗ</EXTERNAL>
        </SPAN>
        <p>В БЛОКЕ 1</p>
    </P>
    <CHAPMARK ID="P0047" STRING="HIDDEN_BLOCK_CLOSE" MARKER="00000000000000000000000000000000000000000000000000OQ01AB" HIDDEN="1" LEVEL="8"/>
    <p>МЕЖДУ БЛОКАМИ.</p>
    <CHAPMARK ID="P01005" STRING="HIDDEN_BLOCK_OPEN#Информация об изменяющих документах" MARKER="00000000000000000000000000000000000000000000000003U4E8OT" HIDDEN="1" LEVEL="8"/>
    <P ID="P0007" CLASS="formattext topleveltext" ALIGN="justify">
        <SPAN class="comment">
            Документ с изменениями, внесенными:<BR/>
            <EXTERNAL ND="499012712" CONTEXT="0000000000000000000000000000000000000000000000000064U0IK">Федеральным законом от 5 августа 2000 года N 118-ФЗ</EXTERNAL> (изменения вступили в силу с 1 января 2002 года);<BR/>
            <EXTERNAL ND="901777906" CONTEXT="0000000000000000000000000000000000000000000000000064U0IK">Федеральным законом от 29 декабря 2000 года N 166-ФЗ</EXTERNAL> (Российская газета, N 248, 31.12.2000) (вступил в силу с 1 января 2001 года);<BR/>
            <EXTERNAL ND="901788506">Федеральным законом от 30 мая 2001 года N 71-ФЗ</EXTERNAL> (Российская газета, N 105, 02.06.2001);<BR/>
            <EXTERNAL ND="901794410" CONTEXT="0000000000000000000000000000000000000000000000000064U0IK">Федеральным законом от 7 августа 2001 года N 118-ФЗ</EXTERNAL>
            <EXTERNAL ND="901794410" CONTEXT="0000000000000000000000000000000000000000000000000064U0IK">Федеральным законом от 7 августа 2001 года N 118-ФЗ</EXTERNAL>
        </SPAN>
        <p>В БЛОКЕ 2.</p>
    </P>
    <CHAPMARK ID="P20047" STRING="HIDDEN_BLOCK_CLOSE" MARKER="00000000000000000000000000000000000000000000000000OQ01AB" HIDDEN="1" LEVEL="8"/>
    <P ID="P0006" CLASS="formattext topleveltext" ALIGN="justify">
        <SPAN class="comment">12121212</SPAN>
    </P>
    <p>ПОСЛЕ БЛОКА 2.</p>
    <CHAPMARK ID="P0005" STRING="HIDDEN_BLOCK_OPEN#Информация об изменяющих документах" MARKER="00000000000000000000000000000000000000000000000003U4E8OT" HIDDEN="1" LEVEL="8"/>
    <p>ПОСЛЕ БЛОКА 2 И ПОСЛЕ HIDDEN_BLOCK_OPEN БЕЗ ЗАКРЫТИЯ.</p>


    <SPAN class="comment">_ПОЧЕМУ НЕ УДАЛ___________________________________________________________________</SPAN>
</root>
"""
# HIDDEN_BLOCK_OPEN атрибут
# Парсинг XML с помощью BeautifulSoup
soup = BeautifulSoup(xml_content, 'xml')


for span in soup.find_all('SPAN'):
        span.decompose()
        
hidden_block_open_tags = soup.find_all('CHAPMARK', {'STRING': re.compile(r'HIDDEN_BLOCK_OPEN')})

for open_tag in hidden_block_open_tags:
    close_tag = open_tag.find_next('CHAPMARK', {'STRING': re.compile(r'HIDDEN_BLOCK_CLOSE')})
    
    if close_tag:
        current = open_tag.next_sibling
        while current and current != close_tag:
            next_sibling = current.next_sibling
            current.extract()
            current = next_sibling

text = ' '.join(soup.stripped_strings)
print(text)

In [ ]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import re

def extract_text_from_xml(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()
    soup = BeautifulSoup(content, 'lxml')

    for table in soup.find_all('table'):
        table.decompose()
    for img in soup.find_all('picture'):
        img.decompose()

    for span in soup.find_all('span'):
        span.decompose()

    hidden_block_open_tags = soup.find_all('chapmark', {'string': re.compile(r'HIDDEN_BLOCK_OPEN')})
    for open_tag in hidden_block_open_tags:
        
        close_tag = open_tag.find_next('chapmark', {'string': re.compile(r'HIDDEN_BLOCK_CLOSE')})
    
        if close_tag:
            current = open_tag.next_sibling
            while current and current != close_tag:
                    next_sibling = current.next_sibling
                    current.extract()
                    current = next_sibling

    # Extract text
    text = ' '.join(soup.stripped_strings)
    cleaned_text = re.sub(r'(--\S+|__\S+|табли[а-я]*|табл.[0-9]*)', '', text, flags=re.IGNORECASE)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text if cleaned_text else None

extracted_path = './work_data/test'

data = []

file_list = sorted(os.listdir(extracted_path))

for file_name in file_list:
    if file_name.endswith('.txt'):
        file_path = os.path.join(extracted_path, file_name)
        try:
            text = extract_text_from_xml(file_path)
            if text is not None:
                data.append({'FileName': file_name, 'Text': text})
        except Exception as e:
            print(f"Error processing file {file_name}: {e}")

dflxml = pd.DataFrame(data)
print(dflxml)

In [ ]:
dflxml['Text'][0]

In [ ]:
dflxml['FileName'] = dflxml['FileName'].str.split('_').str[0]
dflxml

In [ ]:
dflxml['FileName'].nunique()

In [ ]:
duplicate = dflxml[dflxml.duplicated('FileName', keep=False)]
duplicate

In [ ]:
dflxml['Text'][7209]

In [ ]:
# dflxml = dflxml.drop([604, 606, 608,776])
dflxml = dflxml.drop([98,139,3430,3783,6282,6331,6352,6356,6361,6417,6419,6421,6684,6692,7015,7017,7019,7021,7023,7025,7034
                     ,7036,7190,7209])

In [ ]:
dflxml

In [ ]:
dflxml = dflxml.reset_index(drop=True)
dflxml

In [ ]:
dflxml['FileName'].nunique()

In [ ]:
dflxml = dflxml.rename(columns={'FileName': 'iddoc'})
dflxml

In [ ]:
dflxml.to_pickle('./my_df/dflxml_test.pkl')

In [ ]:
import pandas as pd
dflxml=pd.read_pickle("./my_df/dflxml.pkl")
# dflxml=dflxml.sample(n=200)
dflxml

In [ ]:
dflxml['Text'][124]

## Отдельная токенизация

In [7]:
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import time
import torch
import re
import gc

# def remove_punctuation_except_dots(text):
#     return re.sub(r'[^\w\s\.]', '', text)
# dflxml['Text'] = dflxml['Text'].apply(remove_punctuation_except_dots)

texts = new_name_df['combined_vector'].tolist()

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
encoded_inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True, max_length=512)# среднее значение брат 
torch.save(encoded_inputs, './my_df/encoddernew_name_dfcombined_vector.pt')
del encoded_inputs, tokenizer,texts
gc.collect()

C:\Users\Waple\.conda\envs\tf1\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


0

In [ ]:
import re

def clean_text(text):
    return re.sub(r'[^\w\s\.]', '', text)

text = "Hello, world! This is a test: 1234. Numbers      1 should be removed. Special chars like @#$% should go."
cleaned_text = clean_text(text)
cleaned_text = ' '.join(cleaned_text.split())

print(cleaned_text)


In [ ]:
import torch
encoded_inputs = torch.load('./my_df/encoddernew_name_df1.pt')

## Создание векторов с помощью трансформера

In [ ]:
import gc
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import time
import pandas as pd
start_time = time.time()

# torch.cuda.empty_cache()
# torch.backends.cuda.max_split_size_mb = 4

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")

# Загрузка модели на устройство
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased").to(device)
batch_size = 10
output_file = "./my_df/new_name_dfvec1.pkl"

# model = AutoModel.from_pretrained("law-ai/InLegalBERT").to(device)
# batch_size = 10
# output_file = "./my_df/InLegalBERT.pkl"

with tqdm(total=len(encoded_inputs["input_ids"]), desc="Processing texts") as pbar:
    for i in range(0, len(encoded_inputs["input_ids"]), batch_size):
        batch_inputs = {k: torch.tensor(v[i:i+batch_size]).to(device) for k, v in encoded_inputs.items()}
#         batch_inputs = {k: v[i:i+batch_size].clone().detach().to(device) for k, v in encoded_inputs.items()}

        
        with torch.no_grad():
            output = model(**batch_inputs)

        pooler_outputs = [po.cpu().detach().numpy() for po in output.pooler_output]

        iddocs = new_name_df['iddoc'].iloc[i:i+batch_size].tolist()
        df_batch = pd.DataFrame({'iddoc': iddocs, 'namevecnew': pooler_outputs})
        if i == 0:
            df_batch.to_pickle(output_file)
        else:
            existing_df = pd.read_pickle(output_file)
            new_df = pd.concat([existing_df, df_batch])
            new_df.to_pickle(output_file)
            del existing_df, new_df

        del batch_inputs, output, pooler_outputs, df_batch
        gc.collect()

        pbar.update(batch_size)
        print(i)
        print(time.ctime())
end_time = time.time()
print(f"Model processing time: {end_time - start_time} seconds")

## Токенизация и создание векторов с помощью трансформера(max_length>512 -> avg)

In [12]:
import gc
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import time
import pandas as pd
import re

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using device: {device}")
print(time.ctime())

texts = new_name_df['combined_vector'].tolist()

tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
model = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased").to('cuda')

batch_size = 30
output_file_avg = "./my_df2/trainmyname1avg.pkl"
output_file_median = "./my_df2/trainmyname1median.pkl"
max_length = 512

def process_text(text, tokenizer, model, max_length):
    encoded_inputs_list = []

    # Разбиваем текст на части по 512 токенов
    for i in range(0, len(text), max_length):
        part_text = text[i:i + max_length]
        encoded_inputs = tokenizer(part_text, return_tensors="pt", padding='max_length', truncation=True, max_length=512)
        encoded_inputs_list.append(encoded_inputs)

    del encoded_inputs, part_text
    gc.collect()

    part_outputs = []
    for inputs in encoded_inputs_list:
#         inputs = {k: v.to('cuda') for k, v in inputs.items()}
        inputs = {k: v.clone().detach().to(device) for k, v in inputs.items()}
        with torch.no_grad():
            output = model(**inputs)
#             part_outputs.append(output.pooler_output)
            part_outputs.append(output.pooler_output.clone().detach())
    
    del inputs, output
    gc.collect()

    avg_pooler_output = torch.mean(torch.stack(part_outputs), dim=0)
    median_pooler_output = torch.median(torch.stack(part_outputs), dim=0).values
    
    del part_outputs, encoded_inputs_list
    gc.collect()
    
    return avg_pooler_output, median_pooler_output

start_time = time.time()
all_pooler_outputs_avg = []
all_pooler_outputs_median = []

with tqdm(total=len(texts), desc="Processing texts") as pbar:
    for start_idx in range(0, len(texts), batch_size):
        batch_texts = texts[start_idx:start_idx + batch_size]
        batch_pooler_outputs_avg = []
        batch_pooler_outputs_median = []

        for text in batch_texts:
            avg_pooler_output, median_pooler_output = process_text(text, tokenizer, model, max_length)
            batch_pooler_outputs_avg.append(avg_pooler_output.cpu().detach().numpy())
            batch_pooler_outputs_median.append(median_pooler_output.cpu().detach().numpy())

        iddocs = new_name_df['iddoc'].iloc[start_idx:start_idx + batch_size].tolist()
        df_batch_avg = pd.DataFrame({'iddoc': iddocs, 'pooler_output_avg': batch_pooler_outputs_avg})
        df_batch_median = pd.DataFrame({'iddoc': iddocs, 'pooler_output_median': batch_pooler_outputs_median})
        
        if start_idx == 0:
            df_batch_avg.to_pickle(output_file_avg)
            df_batch_median.to_pickle(output_file_median)
        else:
            existing_df_avg = pd.read_pickle(output_file_avg)
            existing_df_median = pd.read_pickle(output_file_median)
            new_df_avg = pd.concat([existing_df_avg, df_batch_avg])
            new_df_median = pd.concat([existing_df_median, df_batch_median])
            new_df_avg.to_pickle(output_file_avg)
            new_df_median.to_pickle(output_file_median)
            del existing_df_avg, new_df_avg, existing_df_median, new_df_median

        del batch_texts, batch_pooler_outputs_avg, batch_pooler_outputs_median, df_batch_avg, df_batch_median
        gc.collect()

        pbar.update(batch_size)
        print(start_idx)
        print(time.ctime())

end_time = time.time()
print(f"Model processing time: {end_time - start_time} seconds")


Using device: cuda
Wed Jul 31 18:49:03 2024


C:\Users\Waple\.conda\envs\tf1\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializi

0
Wed Jul 31 18:49:16 2024


Processing texts:   1%|▍                                                             | 60/9694 [00:15<41:40,  3.85it/s]

30
Wed Jul 31 18:49:23 2024


...

9630
Wed Jul 31 19:25:29 2024


Processing texts: 100%|███████████████████████████████████████████████████████████▉| 9690/9694 [36:28<00:00,  4.34it/s]

9660
Wed Jul 31 19:25:35 2024


Processing texts: 9720it [36:29,  4.44it/s]                                                                            

9690
Wed Jul 31 19:25:37 2024
Model processing time: 2189.714115858078 seconds


## New Name

In [18]:
new_name_df=pd.read_csv('./work_data/name_task_1_part_2.csv',sep=";",encoding='windows-1251',quotechar="'",)
new_name_df = new_name_df.rename(columns={'id': 'iddoc'})

new_name_df

,iddoc,name,1186
0,140004,МУК 4.1.656-96 Методические указания по газохр...,NaN
1,140005,Санитарные правила для предприятий по обработк...,NaN
2,190074,"Европейское Соглашение, касающееся работы экип...",NaN
3,190084,"Конвенция об указании веса тяжелых грузов, пер...",NaN
4,190105,Конвенция об оплачиваемых отпусках (пересмотре...,Конвенция 1970 года об оплачиваемых отпусках (...
...,...,...,...
9689,130668748,НД № 2-020101-176 Правила классификации и пост...,NaN
9690,130671048,О внесении изменений в некоторые акты Правител...,Особенности проведения проверок правил формиро...
9691,130674299,О внесении изменений в Порядок формирования и ...,NaN
9692,130676021,"Об утверждении профессионального стандарта ""Сп...",Деятельность по организации и проведению конгр...


In [19]:
import pandas as pd
import numpy as np

new_name_df['combined_vector'] = new_name_df[['name', '1186']].fillna('').agg(lambda x: '.'.join(filter(None, x)), axis=1)
new_name_df.rename(columns={'id': 'iddoc'}, inplace=True)
new_name_df['combined_vector']

0       МУК 4.1.656-96 Методические указания по газохр...
1       Санитарные правила для предприятий по обработк...
2       Европейское Соглашение, касающееся работы экип...
3       Конвенция об указании веса тяжелых грузов, пер...
4       Конвенция об оплачиваемых отпусках (пересмотре...
                              ...                        
9689    НД № 2-020101-176 Правила классификации и пост...
9690    О внесении изменений в некоторые акты Правител...
9691    О внесении изменений в Порядок формирования и ...
9692    Об утверждении профессионального стандарта "Сп...
9693    Об утверждении профессионального стандарта "Сп...
Name: combined_vector, Length: 9694, dtype: object

In [20]:
trainmynameavg = pd.read_pickle("./my_df2/trainmyname1avg.pkl")
trainmynameavg['pooler_output_avg'] = trainmynameavg['pooler_output_avg'].apply(lambda x: x.reshape(768))
trainmynameavg = trainmynameavg.reset_index(drop=True)

trainmynameavg

,iddoc,pooler_output_avg
0,140004,"[0.3533614, -0.24332023, 0.05609652, 0.1153601..."
1,140005,"[0.32593527, -0.23999172, 0.28076676, 0.152409..."
2,190074,"[0.08047133, -0.039777726, 0.06488275, 0.35665..."
3,190084,"[0.39877644, -0.37896392, 0.3478066, -0.059260..."
4,190105,"[0.15064745, -0.27369267, 0.13626704, 0.016533..."
...,...,...
9689,130668748,"[0.22188985, -0.40385714, 0.18229052, -0.12515..."
9690,130671048,"[0.06462571, -0.029448021, 0.19465742, 0.20484..."
9691,130674299,"[0.14669056, -0.11798106, 0.15181158, 0.096872..."
9692,130676021,"[0.038171273, -0.004897422, 0.039102685, 0.258..."


## Добавим новые признаки organ and name(train dataset)

In [26]:
organ_df=pd.read_csv('./work_data/organ.csv',sep=";",encoding='cp1251')
organ_df = organ_df.rename(columns={'Organ': 'organ'})
organ_df.drop(columns=['idorgan'], inplace=True)

In [27]:
name_df=pd.read_csv('./work_data/name.csv',sep=";",encoding='cp1251')
name_df.drop(columns=['Unnamed: 2', 'Unnamed: 3','Unnamed: 4'], inplace=True)

In [28]:
name_df['iddoc'] = name_df['iddoc'].astype(str)
organ_df['iddoc'] = organ_df['iddoc'].astype(str)
dffull_train = pd.merge(name_df, organ_df, on='iddoc', how='inner')

In [29]:
df_label_train = pd.read_csv('./work_data/DS_TechReg-140624.csv', encoding="windows-1251",sep=';')
df_label_train = df_label_train.rename(columns={'ID': 'iddoc'})

In [30]:
df_label_train['iddoc'] = df_label_train['iddoc'].astype(str)
dffull_train = pd.merge(dffull_train, df_label_train, on='iddoc', how='inner')

In [31]:
trainorganvec = pd.read_pickle("./my_df/trainorganvec.pkl")
trainorganvec = trainorganvec.rename(columns={'namevec': 'trainorganvec'})

In [32]:
trainnamevec = pd.read_pickle("./my_df/trainnamevec.pkl")
trainnamevec = trainnamevec.rename(columns={'namevec': 'trainnamevec'})

In [33]:
trainnamevec['iddoc'] = trainnamevec['iddoc'].astype(str)
trainorganvec['iddoc'] = trainorganvec['iddoc'].astype(str)
trainnameorganvec = pd.merge(trainnamevec, trainorganvec, on='iddoc', how='inner')
dffull_train = pd.merge(dffull_train, trainnameorganvec, on='iddoc', how='inner')

In [34]:
trainruupunkavg = pd.read_pickle("./my_df/trainruupunkavg.pkl")
trainruupunkavg['iddoc'] = trainruupunkavg['iddoc'].astype(str)
trainruupunkavg = trainruupunkavg.rename(columns={'pooler_output': 'vect'})
trainruupunkavg['vect'] = trainruupunkavg['vect'].apply(lambda x: x.reshape(768))
dffull_train = pd.merge(dffull_train, trainruupunkavg, on='iddoc', how='inner')

In [35]:
dffull_train.to_pickle('./my_df/dffull_train.pkl')
dffull_train

,iddoc,name,organ,Label,trainnamevec,trainorganvec,vect
0,1400051,'Санитарные правила для предприятий по обработ...,Заместитель главного государственного санитарн...,Санитарные нормы и правила,"[0.3289106, -0.3153046, 0.19357339, 0.20849438...","[0.37863722, -0.12031019, 0.20947607, 0.258135...","[0.12359218, -0.16039869, 0.12760171, 0.029699..."
1,5200228,'ГОСТ 12.2.061-81 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.20333531, -0.18829913, 0.022558536, 0.32277...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.10657137, -0.14859758, 0.11362507, 0.030690..."
2,5200275,'ГОСТ 12.1.008-76 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.23374136, -0.20337054, 0.0030277222, 0.2893...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.18051448, -0.19030748, 0.11437332, 0.008232..."
3,5200277,'ГОСТ 12.4.155-85 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.22064964, -0.22460273, 0.042272247, 0.29843...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.10624106, -0.20068084, 0.06754017, 0.087890..."
4,5200292,'ГОСТ 12.1.029-80 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.23646508, -0.158473, 0.022069393, 0.2968053...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.09985344, -0.19155689, 0.051527206, 0.10834..."
...,...,...,...,...,...,...,...
1366,1300338507,'О применении дистанционных технологий при обу...,Минтруд России,Охрана и безопасность труда,"[0.2653267, -0.27229616, 0.077737056, 0.272881...","[0.19772974, -0.12900108, 0.098861955, 0.14717...","[0.026345547, -0.15162246, 0.069065, 0.1603872..."
1367,1300457189,'О внесении изменений в Правила обучения по ох...,Правительство РФ,Охрана и безопасность труда,"[0.23206042, -0.27537432, 0.06759465, 0.217285...","[0.20131747, -0.14124425, 0.16144168, 0.097356...","[0.05934743, -0.1224509, 0.07805726, 0.1610681..."
1368,1300583925,'Об организации обучения требованиям охраны тр...,Минтруд России,Охрана и безопасность труда,"[0.23494183, -0.32122934, 0.11517173, 0.434092...","[0.19772974, -0.12900108, 0.098861955, 0.14717...","[0.048510328, -0.10868763, 0.06287158, 0.18918..."
1369,1300598905,'Об аттестации в области промышленной безопасн...,Правительство РФ,Промышленная безопасность,"[0.25426373, -0.19592108, 0.072047524, 0.48183...","[0.20131747, -0.14124425, 0.16144168, 0.097356...","[0.059145734, -0.03285611, 0.065415256, 0.1477..."


## Добавим новые признаки organ and name(test dataset)

In [36]:
organ_df=pd.read_csv('./work_data/organ.csv',sep=";",encoding='cp1251')
organ_df = organ_df.rename(columns={'Organ': 'organ'})
organ_df.drop(columns=['idorgan'], inplace=True)

In [37]:
name_df=pd.read_csv('./work_data/name.csv',sep=";",encoding='cp1251')
name_df.drop(columns=['Unnamed: 2', 'Unnamed: 3','Unnamed: 4'], inplace=True)

In [38]:
name_df['iddoc'] = name_df['iddoc'].astype(str)
organ_df['iddoc'] = organ_df['iddoc'].astype(str)

In [39]:
df_fulltest = pd.merge(name_df, organ_df, on='iddoc', how='inner')

In [40]:
df_label_test = pd.read_csv('./work_data/DS_check_quality.csv', encoding="windows-1251",sep=';')
df_label_test = df_label_test.rename(columns={'ID': 'iddoc'})
df_label_test['iddoc'] = df_label_test['iddoc'].astype(str)
df_fulltest = pd.merge(df_fulltest, df_label_test, on='iddoc', how='inner')

In [41]:
testorganvec = pd.read_pickle("./my_df/testorganvec.pkl")
testorganvec = testorganvec.rename(columns={'namevec': 'testorganvec'})

In [42]:
testnamevec = pd.read_pickle("./my_df/testnamevec.pkl")
testnamevec = testnamevec.rename(columns={'namevec': 'testnamevec'})

In [43]:
testorganvec['iddoc'] = testorganvec['iddoc'].astype(str)
testnamevec['iddoc'] = testnamevec['iddoc'].astype(str)

In [44]:
testorgannamevec = pd.merge(testorganvec, testnamevec, on='iddoc', how='inner')
df_fulltest = pd.merge(df_fulltest, testorgannamevec, on='iddoc', how='inner')

In [45]:
testruupunkavg = pd.read_pickle("./my_df/testruupunkavg.pkl")
testruupunkavg['iddoc'] = testruupunkavg['iddoc'].astype(str)
testruupunkavg = testruupunkavg.rename(columns={'pooler_output': 'vect'})
testruupunkavg['vect'] = testruupunkavg['vect'].apply(lambda x: x.reshape(768))
df_fulltest = pd.merge(df_fulltest, testruupunkavg, on='iddoc', how='inner')

In [46]:
df_fulltest.to_pickle('./my_df/df_fulltest.pkl')
df_fulltest

,iddoc,name,organ,Label,testorganvec,testnamevec,vect
0,1400051,'Санитарные правила для предприятий по обработ...,Заместитель главного государственного санитарн...,Санитарные нормы и правила,"[0.3786372, -0.120310254, 0.20947564, 0.258135...","[0.3289096, -0.31530365, 0.19357279, 0.2084960...","[0.12359218, -0.16039869, 0.12760171, 0.029699..."
1,5200086,'ГОСТ 17.5.3.02-90 Охрана природы (ССОП). Земл...,Госкомприроды СССР,Охрана окружающей среды,"[0.1664121, -0.117298484, 0.15260126, 0.138540...","[0.31862393, -0.3411005, 0.1831128, 0.04432468...","[0.1412907, -0.17420973, 0.13162301, 0.0465316..."
2,5200208,'РДС 10-232-94 Порядок проведения сертификации...,Минстрой России,Оценка соответствия,"[0.23798057, -0.13816689, 0.16876133, 0.081330...","[0.16993566, -0.18714105, 0.10934698, 0.321882...","[0.11319316, -0.1464605, 0.0960037, 0.07665943..."
3,5200209,'РДС 10-233-94 Требования к органам по сертифи...,Минстрой России,Оценка соответствия,"[0.23798057, -0.13816689, 0.16876133, 0.081330...","[0.11356673, -0.13251105, -0.017487448, 0.3879...","[0.1253605, -0.17264639, 0.13933165, 0.0150382..."
4,5200228,'ГОСТ 12.2.061-81 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.14245716, -0.12429103, 0.12220717, 0.144069...","[0.20333536, -0.18829906, 0.022558402, 0.32277...","[0.10657137, -0.14859758, 0.11362507, 0.030690..."
...,...,...,...,...,...,...,...
3950,1300583291,'СТО СРО-С 60542960 00065-2022 Объекты использ...,"СРО ""СОЮЗАТОМСТРОЙ""",Промышленная безопасность,"[0.23033588, -0.26847515, 0.11073253, 0.029051...","[0.22904277, -0.17296153, 0.13700251, 0.168385...","[0.07641777, -0.14503886, 0.19336572, -0.13411..."
3951,1300584509,'Санитарные правила пассажирских перевозок на ...,СЖТ СНГ,Санитарные нормы и правила,"[0.23887007, -0.18820181, 0.20838569, 0.092722...","[0.102712706, -0.17922917, -0.048411377, 0.338...","[0.08611536, -0.13373595, 0.2053212, -0.159156..."
3952,1300666186,'СТО 56910145-009-2014 Полотно нетканое геотек...,"ЗАО ""Техполимер""",Охрана и безопасность труда,"[0.20924278, -0.2000265, 0.14009938, 0.0941980...","[0.23160122, -0.23618263, 0.061281107, 0.16934...","[0.039838407, -0.068976656, 0.13506916, 0.0211..."
3953,1300739796,'Р 1323565.2.002-2022 Государственная система ...,Росстандарт,Метрология,"[0.13282306, -0.109270684, 0.13190684, 0.15616...","[0.07561759, -0.07892581, 0.0060531166, 0.2106...","[0.10631539, -0.10371773, 0.19646613, -0.15908..."


## Проверка качества на тестовом файле + своя нейронная сеть

In [47]:
df_fulltest = pd.read_pickle("./my_df/df_fulltest.pkl")
df_fulltest

,iddoc,name,organ,Label,testorganvec,testnamevec,vect
0,1400051,'Санитарные правила для предприятий по обработ...,Заместитель главного государственного санитарн...,Санитарные нормы и правила,"[0.3786372, -0.120310254, 0.20947564, 0.258135...","[0.3289096, -0.31530365, 0.19357279, 0.2084960...","[0.12359218, -0.16039869, 0.12760171, 0.029699..."
1,5200086,'ГОСТ 17.5.3.02-90 Охрана природы (ССОП). Земл...,Госкомприроды СССР,Охрана окружающей среды,"[0.1664121, -0.117298484, 0.15260126, 0.138540...","[0.31862393, -0.3411005, 0.1831128, 0.04432468...","[0.1412907, -0.17420973, 0.13162301, 0.0465316..."
2,5200208,'РДС 10-232-94 Порядок проведения сертификации...,Минстрой России,Оценка соответствия,"[0.23798057, -0.13816689, 0.16876133, 0.081330...","[0.16993566, -0.18714105, 0.10934698, 0.321882...","[0.11319316, -0.1464605, 0.0960037, 0.07665943..."
3,5200209,'РДС 10-233-94 Требования к органам по сертифи...,Минстрой России,Оценка соответствия,"[0.23798057, -0.13816689, 0.16876133, 0.081330...","[0.11356673, -0.13251105, -0.017487448, 0.3879...","[0.1253605, -0.17264639, 0.13933165, 0.0150382..."
4,5200228,'ГОСТ 12.2.061-81 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.14245716, -0.12429103, 0.12220717, 0.144069...","[0.20333536, -0.18829906, 0.022558402, 0.32277...","[0.10657137, -0.14859758, 0.11362507, 0.030690..."
...,...,...,...,...,...,...,...
3950,1300583291,'СТО СРО-С 60542960 00065-2022 Объекты использ...,"СРО ""СОЮЗАТОМСТРОЙ""",Промышленная безопасность,"[0.23033588, -0.26847515, 0.11073253, 0.029051...","[0.22904277, -0.17296153, 0.13700251, 0.168385...","[0.07641777, -0.14503886, 0.19336572, -0.13411..."
3951,1300584509,'Санитарные правила пассажирских перевозок на ...,СЖТ СНГ,Санитарные нормы и правила,"[0.23887007, -0.18820181, 0.20838569, 0.092722...","[0.102712706, -0.17922917, -0.048411377, 0.338...","[0.08611536, -0.13373595, 0.2053212, -0.159156..."
3952,1300666186,'СТО 56910145-009-2014 Полотно нетканое геотек...,"ЗАО ""Техполимер""",Охрана и безопасность труда,"[0.20924278, -0.2000265, 0.14009938, 0.0941980...","[0.23160122, -0.23618263, 0.061281107, 0.16934...","[0.039838407, -0.068976656, 0.13506916, 0.0211..."
3953,1300739796,'Р 1323565.2.002-2022 Государственная система ...,Росстандарт,Метрология,"[0.13282306, -0.109270684, 0.13190684, 0.15616...","[0.07561759, -0.07892581, 0.0060531166, 0.2106...","[0.10631539, -0.10371773, 0.19646613, -0.15908..."


In [48]:
dffull_train = pd.read_pickle("./my_df/dffull_train.pkl")
dffull_train

,iddoc,name,organ,Label,trainnamevec,trainorganvec,vect
0,1400051,'Санитарные правила для предприятий по обработ...,Заместитель главного государственного санитарн...,Санитарные нормы и правила,"[0.3289106, -0.3153046, 0.19357339, 0.20849438...","[0.37863722, -0.12031019, 0.20947607, 0.258135...","[0.12359218, -0.16039869, 0.12760171, 0.029699..."
1,5200228,'ГОСТ 12.2.061-81 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.20333531, -0.18829913, 0.022558536, 0.32277...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.10657137, -0.14859758, 0.11362507, 0.030690..."
2,5200275,'ГОСТ 12.1.008-76 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.23374136, -0.20337054, 0.0030277222, 0.2893...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.18051448, -0.19030748, 0.11437332, 0.008232..."
3,5200277,'ГОСТ 12.4.155-85 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.22064964, -0.22460273, 0.042272247, 0.29843...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.10624106, -0.20068084, 0.06754017, 0.087890..."
4,5200292,'ГОСТ 12.1.029-80 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.23646508, -0.158473, 0.022069393, 0.2968053...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.09985344, -0.19155689, 0.051527206, 0.10834..."
...,...,...,...,...,...,...,...
1366,1300338507,'О применении дистанционных технологий при обу...,Минтруд России,Охрана и безопасность труда,"[0.2653267, -0.27229616, 0.077737056, 0.272881...","[0.19772974, -0.12900108, 0.098861955, 0.14717...","[0.026345547, -0.15162246, 0.069065, 0.1603872..."
1367,1300457189,'О внесении изменений в Правила обучения по ох...,Правительство РФ,Охрана и безопасность труда,"[0.23206042, -0.27537432, 0.06759465, 0.217285...","[0.20131747, -0.14124425, 0.16144168, 0.097356...","[0.05934743, -0.1224509, 0.07805726, 0.1610681..."
1368,1300583925,'Об организации обучения требованиям охраны тр...,Минтруд России,Охрана и безопасность труда,"[0.23494183, -0.32122934, 0.11517173, 0.434092...","[0.19772974, -0.12900108, 0.098861955, 0.14717...","[0.048510328, -0.10868763, 0.06287158, 0.18918..."
1369,1300598905,'Об аттестации в области промышленной безопасн...,Правительство РФ,Промышленная безопасность,"[0.25426373, -0.19592108, 0.072047524, 0.48183...","[0.20131747, -0.14124425, 0.16144168, 0.097356...","[0.059145734, -0.03285611, 0.065415256, 0.1477..."


In [49]:
from sklearn.utils import shuffle

df_fulltest= shuffle(df_fulltest, random_state=42)
df_fulltest_copy = df_fulltest.copy()

dffull_train= shuffle(dffull_train, random_state=42)
dffull_train_copy = dffull_train.copy()

In [50]:
X_name_train1 = pd.DataFrame(dffull_train['trainnamevec'].tolist())
X_name_test1 = pd.DataFrame(df_fulltest['testnamevec'].tolist())

In [51]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

pca = PCA(n_components=30)
X_name_train = pca.fit_transform(X_name_train1)

explained_variance_ratio = pca.explained_variance_ratio_

cumulative_variance = np.cumsum(explained_variance_ratio)

for i, (var, cum_var) in enumerate(zip(explained_variance_ratio, cumulative_variance)):
    print(f"Component {i+1}: Explained Variance = {var:.4f}, Cumulative Explained Variance = {cum_var:.4f}")

Component 1: Explained Variance = 0.7383, Cumulative Explained Variance = 0.7383
Component 2: Explained Variance = 0.0685, Cumulative Explained Variance = 0.8067
Component 3: Explained Variance = 0.0313, Cumulative Explained Variance = 0.8380
Component 4: Explained Variance = 0.0155, Cumulative Explained Variance = 0.8536
Component 5: Explained Variance = 0.0133, Cumulative Explained Variance = 0.8669
Component 6: Explained Variance = 0.0114, Cumulative Explained Variance = 0.8783
Component 7: Explained Variance = 0.0089, Cumulative Explained Variance = 0.8872
Component 8: Explained Variance = 0.0078, Cumulative Explained Variance = 0.8949
Component 9: Explained Variance = 0.0065, Cumulative Explained Variance = 0.9014
Component 10: Explained Variance = 0.0063, Cumulative Explained Variance = 0.9077
Component 11: Explained Variance = 0.0052, Cumulative Explained Variance = 0.9129
Component 12: Explained Variance = 0.0044, Cumulative Explained Variance = 0.9173
Component 13: Explained V

In [52]:
X_name_test = pca.fit_transform(X_name_test1)

In [53]:
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Dense, BatchNormalization, Dropout
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, Callback
import gc
from tensorflow.keras.initializers import GlorotUniform

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_visible_devices(physical_devices[0], 'GPU')
    print("используем GPU")
tf.keras.backend.clear_session()
gc.collect()
start_time=time.time()
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)
dffull_train['Label'] = dffull_train['Label'].apply(lambda x: x.split(', '))
mlbfull = MultiLabelBinarizer()
y_train = mlbfull.fit_transform(dffull_train['Label'])
y_train = pd.DataFrame(y_train, columns=mlbfull.classes_)

df_fulltest['Label'] = df_fulltest['Label'].apply(lambda x: x.split(', '))
y_test = mlbfull.transform(df_fulltest['Label'])
y_test = pd.DataFrame(y_test, columns=mlbfull.classes_)

X_vect_train = pd.DataFrame(dffull_train['vect'].tolist())
X_vect_test = pd.DataFrame(df_fulltest['vect'].tolist())

X_organ_train = pd.DataFrame(dffull_train['organ'])
X_organ_test = pd.DataFrame(df_fulltest['organ'])

# Разделение на обучающую и валидационную выборки для векторных данных и органов
X_vect_train, X_vect_val, X_organ_train, X_organ_val,X_name_train, X_name_val, y_train, y_val = train_test_split(X_vect_train, X_organ_train,X_name_train, y_train, test_size=0.2, random_state=42)


freq_encoding_train = X_organ_train['organ'].value_counts(normalize=True)
X_organ_train['xfull_organ_encoded'] = X_organ_train['organ'].map(freq_encoding_train)
X_train_organ = X_organ_train[['xfull_organ_encoded']].values


X_organ_val['xfull_organ_encoded'] = X_organ_val['organ'].map(freq_encoding_train)
X_organ_val['xfull_organ_encoded'].fillna(X_organ_val['xfull_organ_encoded'].median(), inplace=True)
X_organ_val = X_organ_val[['xfull_organ_encoded']].values

X_organ_test['xfull_organ_encoded'] = X_organ_test['organ'].map(freq_encoding_train)
X_organ_test['xfull_organ_encoded'].fillna(X_organ_test['xfull_organ_encoded'].median(), inplace=True)
X_test_organ = X_organ_test[['xfull_organ_encoded']].values


scaler_vect = StandardScaler()

X_vect_train = scaler_vect.fit_transform(X_vect_train)
X_vect_val = scaler_vect.transform(X_vect_val)
X_vect_test = scaler_vect.transform(X_vect_test)

# Создаем экземпляр StandardScaler для органов
scaler_organ = StandardScaler()

# Применяем scaler к числовым данным органов
X_train_organ = scaler_organ.fit_transform(X_train_organ)
X_organ_val = scaler_organ.transform(X_organ_val)
X_test_organ = scaler_organ.transform(X_test_organ)

# ###
input_vect = Input(shape=(768,))
input_organ = Input(shape=(1,))
input_name = Input(shape=(30,))

merged = Concatenate()([input_vect,input_organ])

dense = Dense(256, activation='relu', kernel_initializer=GlorotUniform(seed=42))(merged)
dense = BatchNormalization()(dense)
dense = Dropout(0.2)(dense)
dense = Dense(512, activation='relu', kernel_initializer=GlorotUniform(seed=42))(merged)
dense = BatchNormalization()(dense)
dense = Dropout(0.2)(dense)
output = Dense(y_test.shape[1], activation='sigmoid')(dense)

model = Model(inputs=[input_vect,input_organ], outputs=output)

early_stopping = EarlyStopping(monitor='val_loss', patience=20, mode='min', verbose=1, restore_best_weights=True)

class BestMetrics(Callback):
    def __init__(self, validation_data, thresholds):
        super(BestMetrics, self).__init__()
        self.validation_data = validation_data
        self.thresholds = thresholds
        self.best_val_loss = np.Inf
        self.best_f1 = 0
        self.best_threshold = 0

    def on_epoch_end(self, epoch, logs=None):
        current_val_loss = logs.get('val_loss')
        
        if current_val_loss < self.best_val_loss:
            self.best_val_loss = current_val_loss
        
        X_val, y_val = self.validation_data
        y_pred_prob = self.model.predict(X_val)
        
        best_f1_epoch = 0
        best_threshold_epoch = 0

        for threshold in self.thresholds:
            y_pred = (y_pred_prob > threshold).astype(int)
            f1 = f1_score(y_val, y_pred, average='micro')
            print(f'Epoch {epoch+1} - Threshold {threshold} - F1 Score: {f1}')
            
            if f1 > best_f1_epoch:
                best_f1_epoch = f1
                best_threshold_epoch = threshold

        print(f'Epoch {epoch+1} - Best Threshold: {best_threshold_epoch} - Best F1 Score: {best_f1_epoch}')

        if best_f1_epoch > self.best_f1:
            self.best_f1 = best_f1_epoch
            self.best_threshold = best_threshold_epoch

# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), 
#               loss='binary_crossentropy', 
#               metrics=['accuracy'])
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]
f1_callback = BestMetrics(validation_data=([X_vect_val,X_organ_val], y_val), thresholds=thresholds)

используем GPU


In [1]:
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# Определение модели
input_vect = Input(shape=(768,))
input_organ = Input(shape=(1,))
input_name = Input(shape=(30,))

merged = Concatenate()([input_vect,input_organ])
dense = Dense(256, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(merged)
dense = BatchNormalization()(dense)
dense = Dropout(0.2)(dense)
dense = Dense(512, activation='relu', kernel_initializer=tf.keras.initializers.GlorotUniform(seed=42))(dense)
dense = BatchNormalization()(dense)
dense = Dropout(0.2)(dense)
output = Dense(y_test.shape[1], activation='sigmoid')(dense)

model = Model(inputs=[input_vect,input_organ], outputs=output)

import gc
from bayes_opt import BayesianOptimization

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_visible_devices(physical_devices[0], 'GPU')
    print("используем GPU")

# Компилируем модель
def compile_model(learning_rate):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])

# Определяем функцию для оптимизации
def train_evaluate(batch_size, learning_rate,patience):
    batch_size = int(batch_size)
    patience = int(patience)

    compile_model(learning_rate)
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=patience, mode='min', verbose=1, restore_best_weights=True)
    f1_callback = BestMetrics(validation_data=([X_vect_val,X_organ_val], y_val), thresholds=[0.3, 0.4, 0.5, 0.6, 0.7])

    history = model.fit([X_vect_train,X_train_organ], y_train, 
                        epochs=1000, 
                        batch_size=batch_size, 
                        validation_data=([X_vect_train,X_train_organ], y_train), 
                        callbacks=[f1_callback, early_stopping],
                        verbose=0)  # Устанавливаем verbose на 0, чтобы не печатать прогресс по эпохам

    # Возвращаем лучший F1 score на валидации
    return f1_callback.best_f1

# Определяем пространство поиска для байесовской оптимизации
pbounds = {'batch_size': (64, 1000),  'learning_rate': (0.001, 0.01),'patience': (5, 30)}

# Инициализируем байесовскую оптимизацию
optimizer = BayesianOptimization(
    f=train_evaluate,
    pbounds=pbounds,
    random_state=42,
)

# Запускаем оптимизацию
optimizer.maximize(
    init_points=10,
    n_iter=17,
)

# Выводим лучшие результаты
print(optimizer.max)

In [54]:
# model = Model(inputs=[input_vect, input_organ], outputs=output)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [55]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_visible_devices(physical_devices[0], 'GPU')
    print("используем GPU")
import numpy as np
import tensorflow as tf
import random
np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)
history = model.fit([X_vect_train,X_train_organ], y_train, 
                    epochs=1000, 
                    batch_size=310,
                    validation_data=([X_vect_val,X_organ_val], y_val),
                    callbacks=[f1_callback, early_stopping])
end_time = time.time()

print(f"Best F1 Score: {f1_callback.best_f1}")
print(f"Best Threshold: {f1_callback.best_threshold}")
print(f"Best val_loss: {f1_callback.best_val_loss}")
print(f"Model processing time: {end_time - start_time} seconds")

используем GPU
Epoch 1/1000
9/9 [==============================] - 0s 3ms/steposs: 0.9962 - accuracy: 0.11
Epoch 1 - Threshold 0.3 - F1 Score: 0.300174520069808
Epoch 1 - Threshold 0.4 - F1 Score: 0.351931330472103
Epoch 1 - Threshold 0.5 - F1 Score: 0.41973094170403585
Epoch 1 - Threshold 0.6 - F1 Score: 0.5076741440377803
Epoch 1 - Threshold 0.7 - F1 Score: 0.56
Epoch 1 - Best Threshold: 0.7 - Best F1 Score: 0.56
4/4 [==============================] - 1s 145ms/step - loss: 0.8727 - accuracy: 0.3485 - val_loss: 0.6267 - val_accuracy: 0.6145
Epoch 2/1000
9/9 [==============================] - 0s 2ms/steposs: 0.6244 - accuracy: 0.62
Epoch 2 - Threshold 0.3 - F1 Score: 0.3223090799759471
Epoch 2 - Threshold 0.4 - F1 Score: 0.40396341463414637
Epoch 2 - Threshold 0.5 - F1 Score: 0.48871442590775266
Epoch 2 - Threshold 0.6 - F1 Score: 0.5924999999999999
Epoch 2 - Threshold 0.7 - F1 Score: 0.6549520766773163
Epoch 2 - Best Threshold: 0.7 - Best F1 Score: 0.6549520766773163
4/4 [============

In [56]:
test_predictions = model.predict([X_vect_test,X_test_organ])
final_f1 = f1_score(y_test, (test_predictions > f1_callback.best_threshold).astype(int), average='micro')
print(f"Final F1 Score on Test Data: {final_f1}")

124/124 [==============================] - 0s 3ms/step
Final F1 Score on Test Data: 0.838358872960949


In [ ]:
# epochs = 1000, batch = 800 # patience 20 organ and vect

## TF-IDF new Name

In [59]:
dffull_train = dffull_train.rename(columns={'trainnamevec': 'namevec'})
df_fulltest = df_fulltest.rename(columns={'testnamevec': 'namevec'})

merged_df_tfidf = pd.concat([dffull_train, df_fulltest]).drop_duplicates(subset=['iddoc']).reset_index(drop=True)
merged_df_tfidf

,iddoc,name,organ,Label,namevec,trainorganvec,vect,testorganvec
0,1400051,'Санитарные правила для предприятий по обработ...,Заместитель главного государственного санитарн...,Санитарные нормы и правила,"[0.3289106, -0.3153046, 0.19357339, 0.20849438...","[0.37863722, -0.12031019, 0.20947607, 0.258135...","[0.12359218, -0.16039869, 0.12760171, 0.029699...",NaN
1,5200228,'ГОСТ 12.2.061-81 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.20333531, -0.18829913, 0.022558536, 0.32277...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.10657137, -0.14859758, 0.11362507, 0.030690...",NaN
2,5200275,'ГОСТ 12.1.008-76 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.23374136, -0.20337054, 0.0030277222, 0.2893...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.18051448, -0.19030748, 0.11437332, 0.008232...",NaN
3,5200277,'ГОСТ 12.4.155-85 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.22064964, -0.22460273, 0.042272247, 0.29843...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.10624106, -0.20068084, 0.06754017, 0.087890...",NaN
4,5200292,'ГОСТ 12.1.029-80 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.23646508, -0.158473, 0.022069393, 0.2968053...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.09985344, -0.19155689, 0.051527206, 0.10834...",NaN
...,...,...,...,...,...,...,...,...
4716,1300583291,'СТО СРО-С 60542960 00065-2022 Объекты использ...,"СРО ""СОЮЗАТОМСТРОЙ""",Промышленная безопасность,"[0.22904277, -0.17296153, 0.13700251, 0.168385...",NaN,"[0.07641777, -0.14503886, 0.19336572, -0.13411...","[0.23033588, -0.26847515, 0.11073253, 0.029051..."
4717,1300584509,'Санитарные правила пассажирских перевозок на ...,СЖТ СНГ,Санитарные нормы и правила,"[0.102712706, -0.17922917, -0.048411377, 0.338...",NaN,"[0.08611536, -0.13373595, 0.2053212, -0.159156...","[0.23887007, -0.18820181, 0.20838569, 0.092722..."
4718,1300666186,'СТО 56910145-009-2014 Полотно нетканое геотек...,"ЗАО ""Техполимер""",Охрана и безопасность труда,"[0.23160122, -0.23618263, 0.061281107, 0.16934...",NaN,"[0.039838407, -0.068976656, 0.13506916, 0.0211...","[0.20924278, -0.2000265, 0.14009938, 0.0941980..."
4719,1300739796,'Р 1323565.2.002-2022 Государственная система ...,Росстандарт,Метрология,"[0.07561759, -0.07892581, 0.0060531166, 0.2106...",NaN,"[0.10631539, -0.10371773, 0.19646613, -0.15908...","[0.13282306, -0.109270684, 0.13190684, 0.15616..."


In [ ]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

# Объединение данных и удаление дубликатов

merged_df_tfidf = pd.concat([dffull_train, df_fulltest]).drop_duplicates(subset=['iddoc']).reset_index(drop=True)

# Извлечение необходимых данных
documents = merged_df_tfidf['name'].tolist()
iddocs = merged_df_tfidf['iddoc'].tolist()
labels = merged_df_tfidf['Label'].tolist()

# Применение TF-IDF векторайзера
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

# Применение TruncatedSVD
n_components = 300
svd = TruncatedSVD(n_components=n_components)
svd_result = svd.fit_transform(tfidf_matrix)

# Создание DataFrame с результатами SVD
svd_result_df = pd.DataFrame(svd_result, columns=[f"component_{i}" for i in range(n_components)])

# Проверка длины данных перед добавлением
svd_result_df['iddoc'] = iddocs
svd_result_df['Label'] = labels

# Печать информации
print(f"Исходный размер TF-IDF матрицы: {tfidf_matrix.shape}")
print(f"Размерность матрицы после TruncatedSVD: {svd_result.shape}")
print("\nРезультаты SVD с идентификаторами документов:")
print(svd_result_df.head())


In [ ]:
svd_result_df_ = pd.DataFrame(svd_result_df)
svd_result_df_

In [ ]:
from sklearn.utils import shuffle

svd_result_df_= shuffle(svd_result_df_, random_state=42)

In [ ]:
from sklearn.model_selection import train_test_split
svd_result_df_train, svd_result_df_test = train_test_split(svd_result_df_, test_size=0.2, random_state=42)

In [ ]:
svd_result_df_train_x = svd_result_df_train.drop(['iddoc', 'Label'], axis=1)
svd_result_df_test_x = svd_result_df_test.drop(['iddoc', 'Label'], axis=1)
from sklearn.preprocessing import MultiLabelBinarizer
# svd_result_df_train['Label'] = svd_result_df_train['Label'].apply(lambda x: x.split(', '))

mlbTfIdfTrain = MultiLabelBinarizer()
yTfIdfTrain = mlbTfIdfTrain.fit_transform(svd_result_df_train['Label'])
yTfIdfTrain = pd.DataFrame(yTfIdfTrain, columns=mlbTfIdfTrain.classes_)

XTfIdfTrain = svd_result_df_train_x

# svd_result_df_test['Label'] = svd_result_df_test['Label'].apply(lambda x: x.split(', '))

yTfIdfTest = mlbTfIdfTrain.transform(svd_result_df_test['Label'])
yTfIdfTest = pd.DataFrame(yTfIdfTest, columns=mlbTfIdfTrain.classes_)

XTfIdfTest = svd_result_df_test_x

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.multioutput import ClassifierChain

X_train = XTfIdfTrain
X_test = XTfIdfTest
y_train = yTfIdfTrain
y_test = yTfIdfTest

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

param_grid = {
    'hidden_layer_sizes': [(256, 512)],
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': [0.01],
    'learning_rate': ['adaptive'],
    'max_iter': [1000],
    'batch_size': [200],
    'momentum': [0.90],
    'learning_rate_init': [0.002],
}

mlp = MLPClassifier()

grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, n_jobs=-1, cv=3, scoring='f1_samples', verbose=1)
grid_search.fit(X_val, y_val)

results = grid_search.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Mean f1_samples score: {mean_score:.4f}")
    print(f"Parameters: {params}")
    print()

print("Лучшие параметры: ", grid_search.best_params_)

best_mlp_params = grid_search.best_params_
mlp_classifier = MLPClassifier(**best_mlp_params)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', ClassifierChain(mlp_classifier))
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print("Классификационный отчет:\n", classification_report(y_test, y_pred))


## Проверка качества на тестовом файле + sklearn

In [2]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

models = []
# models.append(('SVM', MultiOutputClassifier(SVC(C=1.0,  # штраф за ошибки классификации
#     kernel = 'rbf',  # радиально-базисная функция ядра
#     gamma='scale'))))
# models.append(('OvR_LGBM', MultiOutputClassifier(LGBMClassifier(n_estimators=25))))
models.append(('MLP', MultiOutputClassifier(MLPClassifier(
    hidden_layer_sizes = (256,512),
    activation = 'relu',
    solver = 'adam',
    alpha = 0.01,
    learning_rate = 'adaptive',
    max_iter = 1000,
    batch_size = 200,
    momentum = 0.90,
    learning_rate_init = 0.002))))
# models.append(('Bagging', MultiOutputClassifier(BaggingClassifier(n_estimators=25))))
# models.append(('OvR_AdaBoost', MultiOutputClassifier(AdaBoostClassifier(n_estimators=25))))
# estimators = [
#     ('lr', LogisticRegression(max_iter=1000)),
#     ('rf', RandomForestClassifier(n_estimators=25))
# ]
# stacking = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
# models.append(('Stacking', MultiOutputClassifier(stacking)))


# models.append(('Log_Reg', MultiOutputClassifier(LogisticRegression(
#     penalty='l2',
#     C=1.0,
#     solver='lbfgs',
#     max_iter=1000,
#     class_weight='balanced'
# ))))

models.append(('SGD_LogLoss', MultiOutputClassifier(SGDClassifier(loss='log_loss', max_iter=100, learning_rate='constant', eta0=0.01, random_state=42))))
# models.append(('LDA', MultiOutputClassifier(LinearDiscriminantAnalysis())))
# models.append(('KNN', MultiOutputClassifier(KNeighborsClassifier())))
# models.append(('TREE', MultiOutputClassifier(DecisionTreeClassifier())))
# models.append(('NB', MultiOutputClassifier(GaussianNB())))
# models.append(('RF', MultiOutputClassifier(RandomForestClassifier(n_estimators=25))))

# models.append(('OvR_GBM', MultiOutputClassifier(GradientBoostingClassifier(n_estimators=25))))
# models.append(('OvR_XGB', MultiOutputClassifier(XGBClassifier(n_estimators=25, eval_metric='logloss',))))
# models.append(('OvR_CAT', MultiOutputClassifier(CatBoostClassifier(silent=True, n_estimators=25))))
# models.append(('OvR_RF', MultiOutputClassifier(RandomForestClassifier(n_estimators=25))))

In [60]:
def modelEval1(X_train=None, X_eval=None, y_train=None, y_eval=None,test_value=None,eval_id=[True,True,True,True],
              n_fold=5,scoring='f1_micro'):
    global lst_y_predict

    print('Evaluation Function')
    print(f'Cross Validation Activated, n_splits : {n_fold}, scoring metric: {scoring}')
    
    ''' 1. Split Train/Validate <DataFrame> Set Split '''
#     X_train, X_eval, y_train, y_eval = train_test_split(vectors, labels, test_size=test_value, random_state=42)

    ''' 2. All dataframe '''
    
    X_one = pd.concat([X_train,X_eval],axis=0)
    y_one = pd.concat([y_train,y_eval],axis=0)

    
    ''' 4. Cross Validation, Training/Evaluation, one evaluation'''
    lst_res = []; names = []; lst_train = []; lst_eval = []; lst_one = []; lst_res_mean = []; lst_y_predict = []
    if(any(eval_id)):
        for name, model in models:  # cycle through models & evaluate either cv or train/test
            names.append(name)
            
            # Cross Validation Model on Training Se
            if(eval_id[0]):
                t0=time.time()
                kfold = KFold(n_splits=n_fold)
                cv_res = cross_val_score(model,X_train,y_train, cv=kfold, scoring=scoring)
                t1 = time.time()
                lst_res.append(cv_res)
                tt1 = t1-t0 # total time for n_fold cross evaluation
                
            # Evaluate Fit Model on Training Data
            t2 = time.time()
            if(eval_id[1]):
                t2 = time.time()
                res = model.fit(X_train,y_train)
                train_res = f1_score(res.predict(X_train),y_train, average='micro'); lst_train.append(train_res)
            if(eval_id[2]):
                if(eval_id[1] is False):  # If training hasn't been called yet
                    res = model.fit(X_train,y_train)
                eval_res = f1_score(res.predict(X_eval),y_eval, average='micro'); lst_eval.append(eval_res)
                y_predict=model.predict(X_eval)
                lst_y_predict.append(y_predict)
            t3 = time.time()
            tt2 = t3-t2 # total time for training/evaluation train/prediction
            
            # Evaluate model on entire dataset
            if(eval_id[3]):
                t4 = time.time()
                res = model.fit(X_one,y_one)
                one_res = f1_score(res.predict(X_one),y_one, average='micro'); lst_one.append(one_res)
                t5 = time.time()
                tt3 = t5-t4 # total time for training & evaluation on whole dataframe
            
            ''' [out] Verbal Outputs '''
            # Cross Validation / Training / Evaluation Model Evaluation / Section Times
            lst_res_mean.append(cv_res.mean())
            fn1 = cv_res.mean(); fn2 = cv_res.std();
            fn3 = train_res; fn4 = eval_res; fn5 = one_res
            print(f"{name} : {fn1:.3f}({fn2:.3f}) -> {tt1:.2f}s | {fn3:.3f} & {fn4:.3f} -> {tt2:.2f}s | {fn5:.3f} -> {tt3:.2}s")
    s0 = pd.Series(np.array(lst_res_mean),index=names)
    s1 = pd.Series(np.array(lst_train),index=names)
    s2 = pd.Series(np.array(lst_eval),index=names)
    s3 = pd.Series(np.array(lst_one),index=names)
    pdf = pd.concat([s0,s1,s2,s3],axis=1)
    pdf.columns = ['cv_average','train','test','all']
        
    sns.set(style="whitegrid")
    fig,ax = plt.subplots(1,2,figsize=(15,4))
    ax[0].set_title(f'{n_fold} Cross Validation Results')
    sns.boxplot(data=lst_res, ax=ax[0], orient="v",width=0.3)
    ax[0].set_xticklabels(names, rotation=45, ha="right", fontsize=10)
    sns.stripplot(data=lst_res,ax=ax[0], orient='v',color=".3")
    
    sns.heatmap(pdf,ax=ax[1],annot=True,fmt='.3f',cmap='Blues')
    ax[1].set_title('F1 Scores')
    plt.show()

## Проверяем на готовой модели файл name

In [40]:
df_fulltest = pd.read_pickle("./my_df/df_fulltest.pkl")
dffull_train = pd.read_pickle("./my_df/dffull_train.pkl")

In [41]:
from sklearn.utils import shuffle

df_fulltest= shuffle(df_fulltest, random_state=42)
dffull_train= shuffle(dffull_train, random_state=42)
dffull_train

,iddoc,name,organ,Label,trainnamevec,trainorganvec,vect
430,1200127764,'ГОСТ Р 22.2.09-2015 Безопасность в чрезвычайн...,Росстандарт,Промышленная безопасность,"[0.2855563, -0.21450047, -0.04713926, 0.273567...","[0.13282314, -0.109270625, 0.13190684, 0.15616...","[0.12172554, -0.13963664, 0.10546741, 0.116718..."
588,1200194486,'ГОСТ Р 70511-2022 Оценка соответствия. Оценка...,Росстандарт,Оценка соответствия,"[0.2104687, -0.2030449, 0.027465701, 0.1649854...","[0.13282314, -0.109270625, 0.13190684, 0.15616...","[-0.010893639, -0.074213706, 0.059765805, 0.22..."
296,1200039301,'Методические рекомендации по разработке инстр...,Министерство труда и социального развития РФ,Охрана и безопасность труда,"[0.19567186, -0.2711483, 0.031164335, 0.281890...","[0.43245092, -0.38509068, 0.28445598, -0.10898...","[0.11810719, -0.17636447, 0.10695609, -0.02466..."
184,1200005469,'ГОСТ 8.411-81 Государственная система обеспеч...,Госстандарт СССР,Метрология,"[0.15742917, -0.19496186, -0.0123731345, 0.238...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[-0.004232607, -0.107141346, 0.0384272, 0.2198..."
244,1200014561,'ГОСТ 8.118-85 Государственная система обеспеч...,Госстандарт СССР,Метрология,"[0.17407742, -0.1864671, 0.019858504, 0.308225...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.061051276, -0.1535358, 0.08718315, 0.086728..."
...,...,...,...,...,...,...,...
1095,901742613,'Организация санитарно-гигиенического и дезинф...,Госкомсанэпиднадзор России,Санитарные нормы и правила,"[0.2815536, -0.31136575, 0.14645906, 0.2368792...","[0.35630396, -0.18284945, 0.24060962, 0.133622...","[0.14924094, -0.16554461, 0.15193097, 0.001433..."
1130,901868614,'О совершенствовании противотуберкулезных меро...,Минздрав России,Санитарные нормы и правила,"[0.13238095, -0.17949449, -0.0809301, 0.450526...","[0.26503417, -0.13458908, 0.15122692, 0.060389...","[0.10679596, -0.10799509, 0.059806034, 0.02845..."
1294,1200060936,'ГОСТ Р 12.4.239-2007 Система стандартов безоп...,Росстандарт,Охрана и безопасность труда,"[0.17739098, -0.18805346, 0.092756666, 0.28207...","[0.13282314, -0.109270625, 0.13190684, 0.15616...","[0.051642727, -0.110040665, 0.043895114, 0.208..."
860,499010049,"'Об утверждении перечней учреждений, созданных...",МЧС России,Пожарная безопасность,"[-0.0057764584, -0.018996404, -0.047635127, 0....","[0.31532097, -0.18113564, 0.19666392, 0.043992...","[0.23697138, -0.15449947, 0.079502665, 0.05008..."


In [42]:
trainmynameavg = pd.read_pickle("./my_df2/trainmyname1avg.pkl")
trainmynameavg['pooler_output_avg'] = trainmynameavg['pooler_output_avg'].apply(lambda x: x.reshape(768))
trainmynameavg = trainmynameavg.reset_index(drop=True)
trainmynameavg


,iddoc,pooler_output_avg
0,140004,"[0.3533614, -0.24332023, 0.05609652, 0.1153601..."
1,140005,"[0.32593527, -0.23999172, 0.28076676, 0.152409..."
2,190074,"[0.08047133, -0.039777726, 0.06488275, 0.35665..."
3,190084,"[0.39877644, -0.37896392, 0.3478066, -0.059260..."
4,190105,"[0.15064745, -0.27369267, 0.13626704, 0.016533..."
...,...,...
9689,130668748,"[0.22188985, -0.40385714, 0.18229052, -0.12515..."
9690,130671048,"[0.06462571, -0.029448021, 0.19465742, 0.20484..."
9691,130674299,"[0.14669056, -0.11798106, 0.15181158, 0.096872..."
9692,130676021,"[0.038171273, -0.004897422, 0.039102685, 0.258..."


In [43]:
# import numpy as np
# import pandas as pd
# from sklearn.decomposition import PCA
# import matplotlib.pyplot as plt

# pca2 = PCA(n_components=100)
# XnameTrain_pca = pca2.fit_transform(XtransformerTest)

# explained_variance_ratio = pca2.explained_variance_ratio_

# cumulative_variance = np.cumsum(explained_variance_ratio)

# for i, (var, cum_var) in enumerate(zip(explained_variance_ratio, cumulative_variance)):
#     print(f"Component {i+1}: Explained Variance = {var:.4f}, Cumulative Explained Variance = {cum_var:.4f}")

In [44]:
# XtransformerTest = pca2.transform(XtransformerTest)

In [45]:
dffull_train_x = dffull_train.drop('Label', axis=1)
df_fulltest_x = df_fulltest.drop('Label', axis=1)

In [46]:
dffull_train_x

,iddoc,name,organ,trainnamevec,trainorganvec,vect
430,1200127764,'ГОСТ Р 22.2.09-2015 Безопасность в чрезвычайн...,Росстандарт,"[0.2855563, -0.21450047, -0.04713926, 0.273567...","[0.13282314, -0.109270625, 0.13190684, 0.15616...","[0.12172554, -0.13963664, 0.10546741, 0.116718..."
588,1200194486,'ГОСТ Р 70511-2022 Оценка соответствия. Оценка...,Росстандарт,"[0.2104687, -0.2030449, 0.027465701, 0.1649854...","[0.13282314, -0.109270625, 0.13190684, 0.15616...","[-0.010893639, -0.074213706, 0.059765805, 0.22..."
296,1200039301,'Методические рекомендации по разработке инстр...,Министерство труда и социального развития РФ,"[0.19567186, -0.2711483, 0.031164335, 0.281890...","[0.43245092, -0.38509068, 0.28445598, -0.10898...","[0.11810719, -0.17636447, 0.10695609, -0.02466..."
184,1200005469,'ГОСТ 8.411-81 Государственная система обеспеч...,Госстандарт СССР,"[0.15742917, -0.19496186, -0.0123731345, 0.238...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[-0.004232607, -0.107141346, 0.0384272, 0.2198..."
244,1200014561,'ГОСТ 8.118-85 Государственная система обеспеч...,Госстандарт СССР,"[0.17407742, -0.1864671, 0.019858504, 0.308225...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.061051276, -0.1535358, 0.08718315, 0.086728..."
...,...,...,...,...,...,...
1095,901742613,'Организация санитарно-гигиенического и дезинф...,Госкомсанэпиднадзор России,"[0.2815536, -0.31136575, 0.14645906, 0.2368792...","[0.35630396, -0.18284945, 0.24060962, 0.133622...","[0.14924094, -0.16554461, 0.15193097, 0.001433..."
1130,901868614,'О совершенствовании противотуберкулезных меро...,Минздрав России,"[0.13238095, -0.17949449, -0.0809301, 0.450526...","[0.26503417, -0.13458908, 0.15122692, 0.060389...","[0.10679596, -0.10799509, 0.059806034, 0.02845..."
1294,1200060936,'ГОСТ Р 12.4.239-2007 Система стандартов безоп...,Росстандарт,"[0.17739098, -0.18805346, 0.092756666, 0.28207...","[0.13282314, -0.109270625, 0.13190684, 0.15616...","[0.051642727, -0.110040665, 0.043895114, 0.208..."
860,499010049,"'Об утверждении перечней учреждений, созданных...",МЧС России,"[-0.0057764584, -0.018996404, -0.047635127, 0....","[0.31532097, -0.18113564, 0.19666392, 0.043992...","[0.23697138, -0.15449947, 0.079502665, 0.05008..."


In [47]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=20)  # Уменьшаем до 100 компонент


In [48]:
# import numpy as np
# import pandas as pd
# from sklearn.decomposition import PCA
# import matplotlib.pyplot as plt

# pca = PCA(n_components=5)
# XnameTrain_pca = pca.fit_transform(XnameTrain)

# explained_variance_ratio = pca.explained_variance_ratio_

# cumulative_variance = np.cumsum(explained_variance_ratio)

# for i, (var, cum_var) in enumerate(zip(explained_variance_ratio, cumulative_variance)):
#     print(f"Component {i+1}: Explained Variance = {var:.4f}, Cumulative Explained Variance = {cum_var:.4f}")

# plt.figure(figsize=(10, 6))
# plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, alpha=0.6, align='center', label='Individual explained variance')
# plt.step(range(1, len(cumulative_variance) + 1), cumulative_variance, where='mid', label='Cumulative explained variance')
# plt.xlabel('Principal Components')
# plt.ylabel('Explained Variance Ratio')
# plt.legend(loc='best')
# plt.tight_layout()
# plt.show()


In [49]:
dffull_train['Label'] = dffull_train['Label'].apply(lambda x: x.split(', '))

mlbtransformerTrain = MultiLabelBinarizer()
ytransformerTrain = mlbtransformerTrain.fit_transform(dffull_train['Label'])
ytransformerTrain = pd.DataFrame(ytransformerTrain, columns=mlbtransformerTrain.classes_)

# freq_encoding_train = dffull_train['organ'].value_counts(normalize=True)
# dffull_train['xfull_organ_encoded'] = dffull_train['organ'].map(freq_encoding_train)
# XorganTrain = dffull_train[['xfull_organ_encoded']]
# XorganTrain = pd.DataFrame(dffull_train_x['trainorganvec'].tolist())
# Преобразуем 'vect' в DataFrame
# XnameTrain = pd.DataFrame(dffull_train_x['trainnamevec'].tolist())
# XnameTrain = pca.fit_transform(XnameTrain)
# XnameTrain = pd.DataFrame(XnameTrain)

XvectTrain = pd.DataFrame(dffull_train_x['vect'].tolist())


# Объединяем векторный и числовой признаки
# XvectTrain[768] = XorganTrain['xfull_organ_encoded']
# XtransformerTrain = pd.concat([XvectTrain,XnameTrain], axis=1)
XtransformerTrain = XvectTrain

In [25]:
XvectTrain

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.121726,-0.139637,0.105467,0.116718,-0.138442,0.171796,0.086891,0.247419,-0.196890,0.244275,...,0.239344,0.162732,0.172720,0.607553,0.198131,0.164274,0.163796,-0.140672,0.171242,0.087659
1,-0.010894,-0.074214,0.059766,0.223222,0.003726,0.154238,0.019068,0.156855,-0.063713,0.147598,...,0.177975,0.221565,0.161663,0.416273,0.165902,0.188105,0.111185,0.024148,0.138927,0.027045
2,0.118107,-0.176364,0.106956,-0.024669,-0.089009,0.271752,0.141189,0.233489,-0.222650,0.281777,...,0.278163,0.205149,0.192192,0.719188,0.201962,0.262751,0.164834,-0.071327,0.170451,0.135198
3,-0.004233,-0.107141,0.038427,0.219802,-0.077852,0.192271,-0.011471,0.254946,-0.064583,0.198149,...,0.188420,0.138761,0.173365,0.473599,0.120555,0.117501,0.165556,-0.065006,0.169420,0.075558
4,0.061051,-0.153536,0.087183,0.086729,-0.162839,0.279136,0.062300,0.309271,-0.167022,0.296705,...,0.230675,0.229146,0.222634,0.695143,0.189551,0.240718,0.192767,-0.160626,0.224266,0.125205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366,0.149241,-0.165545,0.151931,0.001434,-0.092638,0.222943,0.090030,0.338888,-0.123098,0.227670,...,0.299340,0.199624,0.183652,0.728257,0.186052,0.176734,0.274569,-0.084907,0.225606,0.109070
1367,0.106796,-0.107995,0.059806,0.028456,-0.050119,0.210295,0.072868,0.317302,-0.037756,0.195803,...,0.212416,0.183545,0.156812,0.595112,0.110992,0.109386,0.218040,-0.055425,0.181718,0.077668
1368,0.051643,-0.110041,0.043895,0.208366,-0.020146,0.177437,0.031646,0.235830,-0.091081,0.211307,...,0.167349,0.108359,0.174380,0.361474,0.121572,0.076706,0.163226,0.028289,0.130318,0.043417
1369,0.236971,-0.154499,0.079503,0.050087,-0.088909,0.302036,0.218016,0.203265,-0.208471,0.352824,...,0.258523,0.097886,0.182620,0.715239,0.170742,0.240365,0.231905,-0.021895,0.301403,0.194796


In [50]:
# df_fulltest['Label'] = df_fulltest['Label'].apply(lambda x: x.split(', '))

# mlbtransformerTest = MultiLabelBinarizer()
# ytransformerTest = mlbtransformerTrain.transform(df_fulltest['Label'])
# ytransformerTest = pd.DataFrame(ytransformerTest, columns=mlbtransformerTrain.classes_)

# df_fulltest['xfull_organ_encoded'] = df_fulltest['organ'].map(freq_encoding_train)
# XorganTest = df_fulltest[['xfull_organ_encoded']]

# Преобразуем 'vect' в DataFrame

# XvectTest = pd.DataFrame(df_fulltest_x['vect'].tolist())
XvectTest = pd.DataFrame(trainmynameavg['pooler_output_avg'].tolist())

# XnameTest = pd.DataFrame(df_fulltest_x['testnamevec'].tolist())
# XnameTest = pca.transform(XnameTest)
# XnameTest = pd.DataFrame(XnameTest)

# XorganTest = pd.DataFrame(df_fulltest_x['testorganvec'].tolist())


# Объединяем векторный и числовой признаки
# XvectTest[768] = XorganTest['xfull_organ_encoded']
# XvectTest[768].fillna(XvectTest[768].mean(), inplace=True)
# XtransformerTest = pd.concat([XvectTest,XnameTest], axis=1)
XtransformerTest = XvectTest

In [51]:
XtransformerTest

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.353361,-0.243320,0.056097,0.115360,-0.032791,0.398964,0.301549,0.404506,-0.237792,0.364938,...,0.147442,0.174253,0.112246,0.858475,0.286932,0.286815,0.252810,-0.182178,0.195503,0.093186
1,0.325935,-0.239992,0.280767,0.152410,0.015082,0.282491,0.364922,0.154419,-0.194627,0.162829,...,0.064095,0.280048,0.064118,0.568542,0.047745,0.242388,0.178616,0.015276,0.197861,0.091460
2,0.080471,-0.039778,0.064883,0.356651,0.033170,0.238256,0.223000,0.019053,-0.048515,0.092934,...,0.002017,0.130841,0.069298,0.528059,0.188968,0.192756,-0.035181,0.025941,0.018656,-0.014280
3,0.398776,-0.378964,0.347807,-0.059261,-0.130373,0.290891,0.475221,0.310504,-0.223411,0.426864,...,0.299909,0.261459,0.337460,0.984841,0.320758,0.347120,0.317649,-0.280297,0.190008,0.273208
4,0.150647,-0.273693,0.136267,0.016533,-0.134183,0.321138,0.304455,0.241956,-0.310449,0.476910,...,0.320894,0.150388,0.354280,0.968229,0.286009,0.408255,0.170743,-0.168815,0.142104,0.210360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9689,0.221890,-0.403857,0.182291,-0.125159,-0.238897,0.340836,0.412804,0.321552,-0.414632,0.538610,...,0.335821,0.195817,0.457760,0.918972,0.308688,0.384629,0.185223,-0.159558,0.285970,0.197536
9690,0.064626,-0.029448,0.194657,0.204840,0.137827,0.045111,0.052437,0.104115,-0.071366,-0.065642,...,0.053031,-0.024066,0.065922,0.062281,0.072315,0.004780,0.062291,0.205900,0.046987,0.054727
9691,0.146691,-0.117981,0.151812,0.096872,-0.021042,0.172170,0.152595,0.148357,-0.154452,0.189064,...,0.158616,0.015169,0.101122,0.668805,0.130857,0.149886,0.158873,0.029864,0.067642,-0.056346
9692,0.038171,-0.004897,0.039103,0.258549,0.078052,0.228774,0.225885,0.141474,-0.158111,0.181955,...,0.251964,0.018961,0.257157,0.376767,0.251073,0.166744,0.168914,0.134019,-0.054821,0.059107


In [52]:
XtransformerTrain

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.121726,-0.139637,0.105467,0.116718,-0.138442,0.171796,0.086891,0.247419,-0.196890,0.244275,...,0.239344,0.162732,0.172720,0.607553,0.198131,0.164274,0.163796,-0.140672,0.171242,0.087659
1,-0.010894,-0.074214,0.059766,0.223222,0.003726,0.154238,0.019068,0.156855,-0.063713,0.147598,...,0.177975,0.221565,0.161663,0.416273,0.165902,0.188105,0.111185,0.024148,0.138927,0.027045
2,0.118107,-0.176364,0.106956,-0.024669,-0.089009,0.271752,0.141189,0.233489,-0.222650,0.281777,...,0.278163,0.205149,0.192192,0.719188,0.201962,0.262751,0.164834,-0.071327,0.170451,0.135198
3,-0.004233,-0.107141,0.038427,0.219802,-0.077852,0.192271,-0.011471,0.254946,-0.064583,0.198149,...,0.188420,0.138761,0.173365,0.473599,0.120555,0.117501,0.165556,-0.065006,0.169420,0.075558
4,0.061051,-0.153536,0.087183,0.086729,-0.162839,0.279136,0.062300,0.309271,-0.167022,0.296705,...,0.230675,0.229146,0.222634,0.695143,0.189551,0.240718,0.192767,-0.160626,0.224266,0.125205
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366,0.149241,-0.165545,0.151931,0.001434,-0.092638,0.222943,0.090030,0.338888,-0.123098,0.227670,...,0.299340,0.199624,0.183652,0.728257,0.186052,0.176734,0.274569,-0.084907,0.225606,0.109070
1367,0.106796,-0.107995,0.059806,0.028456,-0.050119,0.210295,0.072868,0.317302,-0.037756,0.195803,...,0.212416,0.183545,0.156812,0.595112,0.110992,0.109386,0.218040,-0.055425,0.181718,0.077668
1368,0.051643,-0.110041,0.043895,0.208366,-0.020146,0.177437,0.031646,0.235830,-0.091081,0.211307,...,0.167349,0.108359,0.174380,0.361474,0.121572,0.076706,0.163226,0.028289,0.130318,0.043417
1369,0.236971,-0.154499,0.079503,0.050087,-0.088909,0.302036,0.218016,0.203265,-0.208471,0.352824,...,0.258523,0.097886,0.182620,0.715239,0.170742,0.240365,0.231905,-0.021895,0.301403,0.194796


In [ ]:
modelEval1(XtransformerTrain,XtransformerTest,ytransformerTrain,ytransformerTest)

## Изменим размерность всего итогового вектора

In [11]:
# import numpy as np
# import pandas as pd
# from sklearn.decomposition import PCA
# import matplotlib.pyplot as plt

# pca1 = PCA(n_components=100)
# XnameTrain_pca = pca1.fit_transform(XtransformerTrain)

# explained_variance_ratio = pca1.explained_variance_ratio_

# cumulative_variance = np.cumsum(explained_variance_ratio)

# for i, (var, cum_var) in enumerate(zip(explained_variance_ratio, cumulative_variance)):
#     print(f"Component {i+1}: Explained Variance = {var:.4f}, Cumulative Explained Variance = {cum_var:.4f}")

Component 1: Explained Variance = 0.5859, Cumulative Explained Variance = 0.5859
Component 2: Explained Variance = 0.1058, Cumulative Explained Variance = 0.6917
Component 3: Explained Variance = 0.0511, Cumulative Explained Variance = 0.7428
Component 4: Explained Variance = 0.0341, Cumulative Explained Variance = 0.7769
Component 5: Explained Variance = 0.0285, Cumulative Explained Variance = 0.8054
Component 6: Explained Variance = 0.0205, Cumulative Explained Variance = 0.8258
Component 7: Explained Variance = 0.0192, Cumulative Explained Variance = 0.8450
Component 8: Explained Variance = 0.0141, Cumulative Explained Variance = 0.8591
Component 9: Explained Variance = 0.0126, Cumulative Explained Variance = 0.8717
Component 10: Explained Variance = 0.0092, Cumulative Explained Variance = 0.8808
Component 11: Explained Variance = 0.0084, Cumulative Explained Variance = 0.8892
Component 12: Explained Variance = 0.0074, Cumulative Explained Variance = 0.8966
Component 13: Explained V

In [19]:
XnameTrain_pca.shape

(1371, 100)

In [13]:
XtransformerTest = pca1.transform(XtransformerTest)
XtransformerTrain = pca1.transform(XtransformerTrain)

## Используем балнсировщик

In [31]:
from sklearn.utils.class_weight import compute_class_weight
from sklearn.neural_network import MLPClassifier

class_weights = {}
for i, class_label in enumerate(mlbtransformerTrain.classes_):
    class_weights[class_label] = compute_class_weight(
        class_weight='balanced',
        classes=[0, 1],
        y=ytransformerTrain.iloc[:, i]
    ).tolist()

models = {}
for i, class_label in enumerate(mlbtransformerTrain.classes_):
    # Создаем модель для текущей метки класса
    rf = MLPClassifier(
    hidden_layer_sizes = (256,512),
    activation = 'relu',
    solver = 'adam',
    alpha = 0.01,
    learning_rate = 'adaptive',
    max_iter = 1000,
    batch_size = 200,
    momentum = 0.90,
    learning_rate_init = 0.002)

    rf.fit(XtransformerTrain, ytransformerTrain.iloc[:, i])  
    
    models[class_label] = rf

y_preds = {}
for class_label in mlbtransformerTrain.classes_:
    y_preds[class_label] = models[class_label].predict(XtransformerTest)
    
for class_label in mlbtransformerTrain.classes_:
    print(f"Распределение предсказаний для {class_label}:")
    print(pd.Series(y_preds[class_label]).value_counts())    

f1_scores = {}
for class_label in mlbtransformerTrain.classes_:
    idx = np.where(mlbtransformerTrain.classes_ == class_label)[0][0]
    f1_scores[class_label] = f1_score(ytransformerTest.iloc[:, idx], y_preds[class_label], average='binary')  # Используем iloc для доступа к столбцу

for class_label, f1 in f1_scores.items():
    print(f"F1 Score для {class_label}: {f1}")

# Общий F1 Score с 'micro' для всей задачи
ytransformerTest_combined = ytransformerTest.values
y_preds_combined = np.array([y_preds[class_label] for class_label in mlbtransformerTrain.classes_]).T
overall_f1 = f1_score(ytransformerTest_combined, y_preds_combined, average='micro')
print(f'Общий F1 Score: {overall_f1}')

Распределение предсказаний для Метрология:
0    3270
1     685
dtype: int64
Распределение предсказаний для Охрана и безопасность труда:
0    2737
1    1218
dtype: int64
Распределение предсказаний для Охрана окружающей среды:
0    3615
1     340
dtype: int64
Распределение предсказаний для Оценка соответствия:
0    3624
1     331
dtype: int64
Распределение предсказаний для Пожарная безопасность:
0    3107
1     848
dtype: int64
Распределение предсказаний для Промышленная безопасность:
0    3778
1     177
dtype: int64
Распределение предсказаний для Санитарные нормы и правила:
0    3493
1     462
dtype: int64
F1 Score для Метрология: 0.8724919093851132
F1 Score для Охрана и безопасность труда: 0.8335974643423137
F1 Score для Охрана окружающей среды: 0.8515151515151514
F1 Score для Оценка соответствия: 0.8389261744966444
F1 Score для Пожарная безопасность: 0.7142857142857142
F1 Score для Промышленная безопасность: 0.6808510638297872
F1 Score для Санитарные нормы и правила: 0.881606765327695

## Без балансировщика

In [53]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

X_train = XtransformerTrain
X_test = XtransformerTest
y_train = ytransformerTrain
# y_test = ytransformerTest

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


param_grid = {
    'hidden_layer_sizes': [(256,512)],
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': [0.01],
    'learning_rate': ['adaptive'],
    'max_iter': [1000],
    'batch_size': [200],
    'momentum': [0.90],
    'learning_rate_init': [0.002],
}

mlp = MLPClassifier()

grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, n_jobs=-1, cv=3, scoring='f1_samples', verbose=1)

grid_search.fit(X_train, y_train)

results = grid_search.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Mean f1_samples score: {mean_score:.4f}")
    print(f"Parameters: {params}")
    print()

print("Лучшие параметры: ", grid_search.best_params_)

# Оценка модели с лучшими параметрами
best_model = grid_search.best_estimator_
# y_pred = best_model.predict(X_test)
# print("Классификационный отчет:\n", classification_report(y_test, y_pred))


Fitting 3 folds for each of 1 candidates, totalling 3 fits
Mean f1_samples score: 0.8352
Parameters: {'activation': 'relu', 'alpha': 0.01, 'batch_size': 200, 'hidden_layer_sizes': (256, 512), 'learning_rate': 'adaptive', 'learning_rate_init': 0.002, 'max_iter': 1000, 'momentum': 0.9, 'solver': 'adam'}

Лучшие параметры:  {'activation': 'relu', 'alpha': 0.01, 'batch_size': 200, 'hidden_layer_sizes': (256, 512), 'learning_rate': 'adaptive', 'learning_rate_init': 0.002, 'max_iter': 1000, 'momentum': 0.9, 'solver': 'adam'}


In [54]:
X_test

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.353361,-0.243320,0.056097,0.115360,-0.032791,0.398964,0.301549,0.404506,-0.237792,0.364938,...,0.147442,0.174253,0.112246,0.858475,0.286932,0.286815,0.252810,-0.182178,0.195503,0.093186
1,0.325935,-0.239992,0.280767,0.152410,0.015082,0.282491,0.364922,0.154419,-0.194627,0.162829,...,0.064095,0.280048,0.064118,0.568542,0.047745,0.242388,0.178616,0.015276,0.197861,0.091460
2,0.080471,-0.039778,0.064883,0.356651,0.033170,0.238256,0.223000,0.019053,-0.048515,0.092934,...,0.002017,0.130841,0.069298,0.528059,0.188968,0.192756,-0.035181,0.025941,0.018656,-0.014280
3,0.398776,-0.378964,0.347807,-0.059261,-0.130373,0.290891,0.475221,0.310504,-0.223411,0.426864,...,0.299909,0.261459,0.337460,0.984841,0.320758,0.347120,0.317649,-0.280297,0.190008,0.273208
4,0.150647,-0.273693,0.136267,0.016533,-0.134183,0.321138,0.304455,0.241956,-0.310449,0.476910,...,0.320894,0.150388,0.354280,0.968229,0.286009,0.408255,0.170743,-0.168815,0.142104,0.210360
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9689,0.221890,-0.403857,0.182291,-0.125159,-0.238897,0.340836,0.412804,0.321552,-0.414632,0.538610,...,0.335821,0.195817,0.457760,0.918972,0.308688,0.384629,0.185223,-0.159558,0.285970,0.197536
9690,0.064626,-0.029448,0.194657,0.204840,0.137827,0.045111,0.052437,0.104115,-0.071366,-0.065642,...,0.053031,-0.024066,0.065922,0.062281,0.072315,0.004780,0.062291,0.205900,0.046987,0.054727
9691,0.146691,-0.117981,0.151812,0.096872,-0.021042,0.172170,0.152595,0.148357,-0.154452,0.189064,...,0.158616,0.015169,0.101122,0.668805,0.130857,0.149886,0.158873,0.029864,0.067642,-0.056346
9692,0.038171,-0.004897,0.039103,0.258549,0.078052,0.228774,0.225885,0.141474,-0.158111,0.181955,...,0.251964,0.018961,0.257157,0.376767,0.251073,0.166744,0.168914,0.134019,-0.054821,0.059107


In [55]:
y_pred = best_model.predict(X_test)
y_pred

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 1, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0]])

In [56]:
y_pred = pd.DataFrame(y_pred, columns=mlbtransformerTrain.classes_)
y_pred.insert(0, 'iddoc', trainmynameavg['iddoc'])
y_pred

,iddoc,Метрология,Охрана и безопасность труда,Охрана окружающей среды,Оценка соответствия,Пожарная безопасность,Промышленная безопасность,Санитарные нормы и правила
0,140004,0,0,0,0,0,1,0
1,140005,0,0,0,0,0,0,0
2,190074,0,0,0,0,1,0,0
3,190084,0,1,0,0,0,0,0
4,190105,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
9689,130668748,0,0,0,0,1,0,0
9690,130671048,0,1,0,0,0,0,0
9691,130674299,0,1,0,0,1,0,0
9692,130676021,0,0,0,0,0,0,1


In [23]:
y_pred = best_model.predict(X_test)
y_pred

array([[0, 1, 0, ..., 0, 1, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [76]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.multioutput import ClassifierChain

# Ваши данные (замените XtransformerTrain и другие переменные на ваши данные)
X_train = XtransformerTrain
X_test = XtransformerTest
y_train = ytransformerTrain
y_test = ytransformerTest

# Разделение на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# Параметры для MLPClassifier
param_grid = {
    'hidden_layer_sizes': [(256, 512)],
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': [0.01],
    'learning_rate': ['adaptive'],
    'max_iter': [1000],
    'batch_size': [200],
    'momentum': [0.90],
    'learning_rate_init': [0.002],
}

# Инициализация MLPClassifier
mlp = MLPClassifier()

# Поиск лучших параметров с помощью GridSearchCV
grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, n_jobs=-1, cv=3, scoring='f1_samples', verbose=1)
grid_search.fit(X_val, y_val)

# Вывод результатов GridSearchCV
results = grid_search.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Mean f1_samples score: {mean_score:.4f}")
    print(f"Parameters: {params}")
    print()

print("Лучшие параметры: ", grid_search.best_params_)

# Использование лучших параметров для MLPClassifier в пайплайне
best_mlp_params = grid_search.best_params_
mlp_classifier = MLPClassifier(**best_mlp_params)

# Пайплайн с масштабированием и MLPClassifier
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Масштабирование признаков
    ('clf', ClassifierChain(mlp_classifier))
])

# Обучение модели
pipeline.fit(X_train, y_train)

# Оценка модели
y_pred = pipeline.predict(X_test)

# Оценка качества модели
print("Классификационный отчет:\n", classification_report(y_test, y_pred))

Fitting 3 folds for each of 1 candidates, totalling 3 fits
Mean f1_samples score: 0.7493
Parameters: {'activation': 'relu', 'alpha': 0.01, 'batch_size': 200, 'hidden_layer_sizes': (256, 512), 'learning_rate': 'adaptive', 'learning_rate_init': 0.002, 'max_iter': 1000, 'momentum': 0.9, 'solver': 'adam'}

Лучшие параметры:  {'activation': 'relu', 'alpha': 0.01, 'batch_size': 200, 'hidden_layer_sizes': (256, 512), 'learning_rate': 'adaptive', 'learning_rate_init': 0.002, 'max_iter': 1000, 'momentum': 0.9, 'solver': 'adam'}
Классификационный отчет:
               precision    recall  f1-score   support

           0       0.98      0.75      0.85       860
           1       0.94      0.68      0.79      1306
           2       0.85      0.84      0.84       320
           3       0.76      0.92      0.84       265
           4       0.77      0.73      0.75       622
           5       0.65      0.87      0.74       105
           6       0.91      0.86      0.88       484

   micro avg   

C:\Users\Waple\.conda\envs\tf1\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# TF-IDF(текст)(повторяющие слова важны) SKLEARN ( ЭТО РАЗРЕЖЕННЫЕ ВЕКТОРА - СНИЗИТЬ РАЗМЕРНОСТЬ) РАЗМЕР - СЛОВАРЬ
# 1)УМЕНЬШИТЬ С ПОМОЩЬЮ МЕТОДА ГЛАВНЫХ КОМПОНЕНТ 2)tsne - снижение размерности

# классификатор по количеству классов. 7 классов - 7 классификаторов! подход - разграничить объекты разных классов
# оценка avg mediana
# предобработка - убрать информацию в скобках
# внесение изменений/ поправки
# onvsrest
# размеченные -эксперты

## TF-IDF

## Тестовая выборка и обучающая выборка, merge

In [ ]:
import pandas as pd
dflxml_test=pd.read_pickle("./my_df/dflxml_test.pkl")
df_label_test = pd.read_csv('./work_data/DS_check_quality.csv', encoding="windows-1251",sep=';')
df_label_test = df_label_test.rename(columns={'ID': 'iddoc'})
df_label_test['iddoc'] = df_label_test['iddoc'].astype(str)
svd_result_df_test = pd.merge(dflxml_test, df_label_test, on='iddoc', how='inner')
svd_result_df_test

In [ ]:
import pandas as pd
dflxml=pd.read_pickle("./my_df/dflxml.pkl")
df_label_train = pd.read_csv('./work_data/DS_TechReg-140624.csv', encoding="windows-1251",sep=';')
df_label_train = df_label_train.rename(columns={'ID': 'iddoc'})
df_label_train['iddoc'] = df_label_train['iddoc'].astype(str)
svd_result_df_train = pd.merge(dflxml, df_label_train, on='iddoc', how='inner')
svd_result_df_train

In [ ]:
merged_df = pd.concat([svd_result_df_train, svd_result_df_test]).drop_duplicates(subset=['iddoc']).reset_index(drop=True)
merged_df

In [ ]:
from sklearn.decomposition import TruncatedSVD
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

documents = merged_df['Text'].tolist()
iddocs = merged_df['iddoc'].tolist()
labels = merged_df['Label'].tolist()

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

n_components = 300
svd = TruncatedSVD(n_components=n_components)

svd_result = svd.fit_transform(tfidf_matrix)

svd_result_df = pd.DataFrame(svd_result, columns=[f"component_{i}" for i in range(n_components)])
svd_result_df['iddoc'] = iddocs 
svd_result_df['Label'] = labels 

print(f"Исходный размер TF-IDF матрицы: {tfidf_matrix.shape}")
print(f"Размерность матрицы после TruncatedSVD: {svd_result.shape}")
print("\nРезультаты SVD с идентификаторами документов:")
print(svd_result_df)


In [ ]:
svd_result_df_ = pd.DataFrame(svd_result_df)
svd_result_df_

In [ ]:
svd_result_df_.to_pickle('./my_df/svd_result_df_.pkl')

## Разбиваем тренируем тестируем TF-IDF

In [29]:
import pandas as pd
svd_result_df_=pd.read_pickle("./my_df/svd_result_df_.pkl")

In [33]:
svd_result_df_

,component_0,component_1,component_2,component_3,component_4,component_5,component_6,component_7,component_8,component_9,...,component_292,component_293,component_294,component_295,component_296,component_297,component_298,component_299,iddoc,Label
3253,0.403099,-0.130357,-0.181982,0.012088,0.235052,0.031654,0.216490,0.069092,0.066397,0.021630,...,0.005429,-0.004703,-0.005260,-0.016949,0.010182,0.019051,0.003072,-0.007738,1200111766,Охрана и безопасность труда
3915,0.345562,-0.088414,-0.076634,0.033900,0.102252,0.081538,0.085999,0.080050,0.058832,0.015303,...,-0.017777,-0.035962,0.016128,0.001085,0.005210,0.000854,-0.010230,-0.038463,1200179862,Охрана и безопасность труда
1010,0.087989,0.001714,-0.009829,-0.000595,-0.072693,0.001617,0.024547,-0.006387,-0.039423,0.048394,...,-0.027821,0.003393,-0.008333,-0.009456,0.007399,-0.003623,-0.027543,0.034006,564952848,Санитарные нормы и правила
1954,0.218970,-0.103417,0.130136,-0.108442,0.016945,-0.051842,-0.024397,-0.003283,-0.017326,-0.082118,...,-0.019156,-0.012257,0.016084,-0.007756,-0.026295,-0.062236,0.016214,-0.000601,1200034094,Метрология
1580,0.168056,-0.120589,0.310830,-0.178082,0.057914,-0.132209,0.024819,-0.017237,0.050612,0.208133,...,0.000308,0.007194,-0.009939,-0.016269,0.020976,-0.001280,0.021588,0.016638,1200014281,Метрология
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4426,0.055763,0.042185,0.023547,0.022481,-0.082422,0.003769,0.056741,0.029051,0.002362,0.026137,...,-0.020423,-0.014346,-0.007894,0.015818,0.012452,0.019695,-0.006991,-0.001822,564841211,Санитарные нормы и правила
466,0.189888,-0.031271,-0.000571,0.040602,-0.003063,0.008662,-0.025407,-0.035381,-0.000313,-0.006508,...,-0.003600,0.002631,-0.019891,-0.007799,-0.013320,0.019503,-0.000982,0.011889,1200158281,Оценка соответствия
3092,0.244845,-0.128823,0.180374,-0.075627,0.058227,-0.032483,0.031819,0.014536,0.004131,-0.069624,...,0.022386,0.010381,-0.003084,0.017853,0.021911,-0.024813,0.005229,-0.046637,1200107024,Метрология
3772,0.341267,-0.117824,0.060428,-0.055111,0.064605,-0.047736,0.069848,0.036637,-0.059772,-0.052451,...,0.002240,-0.022956,-0.034948,-0.043040,-0.026133,-0.038814,-0.018084,-0.014448,1200163941,Метрология


In [30]:
from sklearn.utils import shuffle

svd_result_df_= shuffle(svd_result_df_, random_state=42)

In [31]:
from sklearn.model_selection import train_test_split
svd_result_df_train, svd_result_df_test = train_test_split(svd_result_df_, test_size=0.2, random_state=42)

In [32]:
svd_result_df_train_x = svd_result_df_train.drop(['iddoc', 'Label'], axis=1)
svd_result_df_test_x = svd_result_df_test.drop(['iddoc', 'Label'], axis=1)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
svd_result_df_train['Label'] = svd_result_df_train['Label'].apply(lambda x: x.split(', '))

mlbTfIdfTrain = MultiLabelBinarizer()
yTfIdfTrain = mlbTfIdfTrain.fit_transform(svd_result_df_train['Label'])
yTfIdfTrain = pd.DataFrame(yTfIdfTrain, columns=mlbTfIdfTrain.classes_)

XTfIdfTrain = svd_result_df_train_x

In [ ]:
svd_result_df_test['Label'] = svd_result_df_test['Label'].apply(lambda x: x.split(', '))

yTfIdfTest = mlbTfIdfTrain.transform(svd_result_df_test['Label'])
yTfIdfTest = pd.DataFrame(yTfIdfTest, columns=mlbTfIdfTrain.classes_)

XTfIdfTest = svd_result_df_test_x

In [ ]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

X_train = XTfIdfTrain
X_test = XTfIdfTest
y_train = yTfIdfTrain
y_test = yTfIdfTest

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


param_grid = {
    'hidden_layer_sizes': [(64,64)],
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': [0.01],
    'learning_rate': ['adaptive'],
    'max_iter': [1000],
    'batch_size': [200],
    'momentum': [0.90],
    'learning_rate_init': [0.002],
}

mlp = MLPClassifier()

grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, n_jobs=-1, cv=3, scoring='f1_samples', verbose=1)

grid_search.fit(X_train, y_train)

results = grid_search.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Mean f1_samples score: {mean_score:.4f}")
    print(f"Parameters: {params}")
    print()

print("Лучшие параметры: ", grid_search.best_params_)

# Оценка модели с лучшими параметрами
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Классификационный отчет:\n", classification_report(y_test, y_pred))


In [3]:
import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.multioutput import ClassifierChain

X_train = XTfIdfTrain
X_test = XTfIdfTest
y_train = yTfIdfTrain
y_test = yTfIdfTest

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

param_grid = {
    'hidden_layer_sizes': [(256, 512)],
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': [0.01],
    'learning_rate': ['adaptive'],
    'max_iter': [1000],
    'batch_size': [200],
    'momentum': [0.90],
    'learning_rate_init': [0.002],
}

mlp = MLPClassifier()

grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, n_jobs=-1, cv=3, scoring='f1_samples', verbose=1)
grid_search.fit(X_val, y_val)

results = grid_search.cv_results_
for mean_score, params in zip(results['mean_test_score'], results['params']):
    print(f"Mean f1_samples score: {mean_score:.4f}")
    print(f"Parameters: {params}")
    print()

print("Лучшие параметры: ", grid_search.best_params_)

best_mlp_params = grid_search.best_params_
mlp_classifier = MLPClassifier(**best_mlp_params)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', ClassifierChain(mlp_classifier))
])

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

print("Классификационный отчет:\n", classification_report(y_test, y_pred))


In [ ]:
modelEval1(XTfIdfTrain,XTfIdfTest,yTfIdfTrain,yTfIdfTest)

## Векторизовали name

In [20]:
# файл вектора с именами для обучения
new_name_dfvec1 = pd.read_pickle("./my_df2/combined_vectortrainname55kavg.pkl")
new_name_dfvec1 = new_name_dfvec1.rename(columns={'pooler_output_avg': 'pooler_output_avg_actual'})
new_name_dfvec1 = new_name_dfvec1.reset_index(drop=True)
new_name_dfvec1['pooler_output_avg_actual'] = new_name_dfvec1['pooler_output_avg_actual'].apply(lambda x: x.reshape(768))
new_name_dfvec1

,iddoc,pooler_output_avg_actual
0,1400040,"[0.3533614, -0.24332023, 0.05609652, 0.1153601..."
1,1400051,"[0.32593527, -0.23999172, 0.28076676, 0.152409..."
2,5200025,"[-0.0014489879, 0.051369514, -0.053475883, 0.4..."
3,5200058,"[0.20856021, -0.37045068, 0.12602767, -0.06559..."
4,5200086,"[0.46679097, -0.3831445, 0.3779671, -0.3678596..."
...,...,...
56510,1306528563,"[0.33210588, -0.3923719, 0.17211705, -0.162469..."
56511,1306588785,"[0.14582644, 0.05809701, 0.017973164, 0.434785..."
56512,1306610181,"[0.18249983, -0.12071254, 0.033566743, 0.15034..."
56513,1306663466,"[0.27992868, -0.047369987, 0.08858715, 0.24100..."


In [21]:
import pandas as pd

# df_fulltest = pd.read_pickle("./my_df/df_fulltest.pkl")
dffull_train = pd.read_pickle("./my_df/dffull_train.pkl")

In [22]:
dffull_train

,iddoc,name,organ,Label,trainnamevec,trainorganvec,vect
0,1400051,'Санитарные правила для предприятий по обработ...,Заместитель главного государственного санитарн...,Санитарные нормы и правила,"[0.3289106, -0.3153046, 0.19357339, 0.20849438...","[0.37863722, -0.12031019, 0.20947607, 0.258135...","[0.12359218, -0.16039869, 0.12760171, 0.029699..."
1,5200228,'ГОСТ 12.2.061-81 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.20333531, -0.18829913, 0.022558536, 0.32277...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.10657137, -0.14859758, 0.11362507, 0.030690..."
2,5200275,'ГОСТ 12.1.008-76 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.23374136, -0.20337054, 0.0030277222, 0.2893...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.18051448, -0.19030748, 0.11437332, 0.008232..."
3,5200277,'ГОСТ 12.4.155-85 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.22064964, -0.22460273, 0.042272247, 0.29843...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.10624106, -0.20068084, 0.06754017, 0.087890..."
4,5200292,'ГОСТ 12.1.029-80 Система стандартов безопасно...,Госстандарт СССР,Охрана и безопасность труда,"[0.23646508, -0.158473, 0.022069393, 0.2968053...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.09985344, -0.19155689, 0.051527206, 0.10834..."
...,...,...,...,...,...,...,...
1366,1300338507,'О применении дистанционных технологий при обу...,Минтруд России,Охрана и безопасность труда,"[0.2653267, -0.27229616, 0.077737056, 0.272881...","[0.19772974, -0.12900108, 0.098861955, 0.14717...","[0.026345547, -0.15162246, 0.069065, 0.1603872..."
1367,1300457189,'О внесении изменений в Правила обучения по ох...,Правительство РФ,Охрана и безопасность труда,"[0.23206042, -0.27537432, 0.06759465, 0.217285...","[0.20131747, -0.14124425, 0.16144168, 0.097356...","[0.05934743, -0.1224509, 0.07805726, 0.1610681..."
1368,1300583925,'Об организации обучения требованиям охраны тр...,Минтруд России,Охрана и безопасность труда,"[0.23494183, -0.32122934, 0.11517173, 0.434092...","[0.19772974, -0.12900108, 0.098861955, 0.14717...","[0.048510328, -0.10868763, 0.06287158, 0.18918..."
1369,1300598905,'Об аттестации в области промышленной безопасн...,Правительство РФ,Промышленная безопасность,"[0.25426373, -0.19592108, 0.072047524, 0.48183...","[0.20131747, -0.14124425, 0.16144168, 0.097356...","[0.059145734, -0.03285611, 0.065415256, 0.1477..."


In [23]:
from sklearn.utils import shuffle

dffull_train= shuffle(dffull_train, random_state=42)
# df_fulltest= shuffle(df_fulltest, random_state=42)

In [24]:
# combined_df = pd.concat([df_fulltest, dffull_train], ignore_index=True)
# dffull_train = combined_df.drop_duplicates(subset='iddoc', keep='first').reset_index(drop=True)
# dffull_train
# совместил метки train test

In [25]:
# # файл тестовый
# trainmynameavg = pd.read_pickle("./my_df2/trainmyname1avg.pkl")
# trainmynameavg['pooler_output_avg'] = trainmynameavg['pooler_output_avg'].apply(lambda x: x.reshape(768))
# trainmynameavg = trainmynameavg.reset_index(drop=True)
# trainmynameavg

In [26]:
# trainmynameavg['iddoc'] = trainmynameavg['iddoc'].astype(str)
new_name_dfvec1['iddoc'] = new_name_dfvec1['iddoc'].astype(str)

dffull_train = pd.merge(dffull_train, new_name_dfvec1, on='iddoc', how='inner')
dffull_train
# файл для обучения готов

,iddoc,name,organ,Label,trainnamevec,trainorganvec,vect,pooler_output_avg_actual
0,1200127764,'ГОСТ Р 22.2.09-2015 Безопасность в чрезвычайн...,Росстандарт,Промышленная безопасность,"[0.2855563, -0.21450047, -0.04713926, 0.273567...","[0.13282314, -0.109270625, 0.13190684, 0.15616...","[0.12172554, -0.13963664, 0.10546741, 0.116718...","[0.29152542, -0.21074589, 0.18180375, 0.038432..."
1,1200194486,'ГОСТ Р 70511-2022 Оценка соответствия. Оценка...,Росстандарт,Оценка соответствия,"[0.2104687, -0.2030449, 0.027465701, 0.1649854...","[0.13282314, -0.109270625, 0.13190684, 0.15616...","[-0.010893639, -0.074213706, 0.059765805, 0.22...","[0.3551837, -0.12282513, 0.10698172, 0.0876703..."
2,1200039301,'Методические рекомендации по разработке инстр...,Министерство труда и социального развития РФ,Охрана и безопасность труда,"[0.19567186, -0.2711483, 0.031164335, 0.281890...","[0.43245092, -0.38509068, 0.28445598, -0.10898...","[0.11810719, -0.17636447, 0.10695609, -0.02466...","[0.14543203, -0.19587837, -0.0039892937, 0.165..."
3,1200005469,'ГОСТ 8.411-81 Государственная система обеспеч...,Госстандарт СССР,Метрология,"[0.15742917, -0.19496186, -0.0123731345, 0.238...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[-0.004232607, -0.107141346, 0.0384272, 0.2198...","[0.19770938, -0.2165873, 0.014208878, 0.133946..."
4,1200014561,'ГОСТ 8.118-85 Государственная система обеспеч...,Госстандарт СССР,Метрология,"[0.17407742, -0.1864671, 0.019858504, 0.308225...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.061051276, -0.1535358, 0.08718315, 0.086728...","[0.2755334, -0.2549058, 0.09994121, 0.12830979..."
...,...,...,...,...,...,...,...,...
1366,901742613,'Организация санитарно-гигиенического и дезинф...,Госкомсанэпиднадзор России,Санитарные нормы и правила,"[0.2815536, -0.31136575, 0.14645906, 0.2368792...","[0.35630396, -0.18284945, 0.24060962, 0.133622...","[0.14924094, -0.16554461, 0.15193097, 0.001433...","[0.24474604, -0.108858354, 0.195402, 0.1332749..."
1367,901868614,'О совершенствовании противотуберкулезных меро...,Минздрав России,Санитарные нормы и правила,"[0.13238095, -0.17949449, -0.0809301, 0.450526...","[0.26503417, -0.13458908, 0.15122692, 0.060389...","[0.10679596, -0.10799509, 0.059806034, 0.02845...","[0.15232515, -0.14815977, 0.0357307, 0.2969926..."
1368,1200060936,'ГОСТ Р 12.4.239-2007 Система стандартов безоп...,Росстандарт,Охрана и безопасность труда,"[0.17739098, -0.18805346, 0.092756666, 0.28207...","[0.13282314, -0.109270625, 0.13190684, 0.15616...","[0.051642727, -0.110040665, 0.043895114, 0.208...","[0.24297185, -0.23234145, 0.127411, 0.04034283..."
1369,499010049,"'Об утверждении перечней учреждений, созданных...",МЧС России,Пожарная безопасность,"[-0.0057764584, -0.018996404, -0.047635127, 0....","[0.31532097, -0.18113564, 0.19666392, 0.043992...","[0.23697138, -0.15449947, 0.079502665, 0.05008...","[0.13283248, -0.09122601, 0.20717815, 0.251213..."


In [27]:
dffull_train_x = dffull_train.drop('Label', axis=1)

In [28]:
dffull_train_x

,iddoc,name,organ,trainnamevec,trainorganvec,vect,pooler_output_avg_actual
0,1200127764,'ГОСТ Р 22.2.09-2015 Безопасность в чрезвычайн...,Росстандарт,"[0.2855563, -0.21450047, -0.04713926, 0.273567...","[0.13282314, -0.109270625, 0.13190684, 0.15616...","[0.12172554, -0.13963664, 0.10546741, 0.116718...","[0.29152542, -0.21074589, 0.18180375, 0.038432..."
1,1200194486,'ГОСТ Р 70511-2022 Оценка соответствия. Оценка...,Росстандарт,"[0.2104687, -0.2030449, 0.027465701, 0.1649854...","[0.13282314, -0.109270625, 0.13190684, 0.15616...","[-0.010893639, -0.074213706, 0.059765805, 0.22...","[0.3551837, -0.12282513, 0.10698172, 0.0876703..."
2,1200039301,'Методические рекомендации по разработке инстр...,Министерство труда и социального развития РФ,"[0.19567186, -0.2711483, 0.031164335, 0.281890...","[0.43245092, -0.38509068, 0.28445598, -0.10898...","[0.11810719, -0.17636447, 0.10695609, -0.02466...","[0.14543203, -0.19587837, -0.0039892937, 0.165..."
3,1200005469,'ГОСТ 8.411-81 Государственная система обеспеч...,Госстандарт СССР,"[0.15742917, -0.19496186, -0.0123731345, 0.238...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[-0.004232607, -0.107141346, 0.0384272, 0.2198...","[0.19770938, -0.2165873, 0.014208878, 0.133946..."
4,1200014561,'ГОСТ 8.118-85 Государственная система обеспеч...,Госстандарт СССР,"[0.17407742, -0.1864671, 0.019858504, 0.308225...","[0.14245707, -0.12429103, 0.122207195, 0.14406...","[0.061051276, -0.1535358, 0.08718315, 0.086728...","[0.2755334, -0.2549058, 0.09994121, 0.12830979..."
...,...,...,...,...,...,...,...
1366,901742613,'Организация санитарно-гигиенического и дезинф...,Госкомсанэпиднадзор России,"[0.2815536, -0.31136575, 0.14645906, 0.2368792...","[0.35630396, -0.18284945, 0.24060962, 0.133622...","[0.14924094, -0.16554461, 0.15193097, 0.001433...","[0.24474604, -0.108858354, 0.195402, 0.1332749..."
1367,901868614,'О совершенствовании противотуберкулезных меро...,Минздрав России,"[0.13238095, -0.17949449, -0.0809301, 0.450526...","[0.26503417, -0.13458908, 0.15122692, 0.060389...","[0.10679596, -0.10799509, 0.059806034, 0.02845...","[0.15232515, -0.14815977, 0.0357307, 0.2969926..."
1368,1200060936,'ГОСТ Р 12.4.239-2007 Система стандартов безоп...,Росстандарт,"[0.17739098, -0.18805346, 0.092756666, 0.28207...","[0.13282314, -0.109270625, 0.13190684, 0.15616...","[0.051642727, -0.110040665, 0.043895114, 0.208...","[0.24297185, -0.23234145, 0.127411, 0.04034283..."
1369,499010049,"'Об утверждении перечней учреждений, созданных...",МЧС России,"[-0.0057764584, -0.018996404, -0.047635127, 0....","[0.31532097, -0.18113564, 0.19666392, 0.043992...","[0.23697138, -0.15449947, 0.079502665, 0.05008...","[0.13283248, -0.09122601, 0.20717815, 0.251213..."


In [29]:
dffull_train['Label'] = dffull_train['Label'].apply(lambda x: x.split(', '))

mlbtransformerTrain = MultiLabelBinarizer()
ytransformerTrain = mlbtransformerTrain.fit_transform(dffull_train['Label'])
ytransformerTrain = pd.DataFrame(ytransformerTrain, columns=mlbtransformerTrain.classes_)
XvectTrain = pd.DataFrame(dffull_train_x['pooler_output_avg_actual'].tolist())
XtransformerTrain = XvectTrain

In [30]:
XvectTest = pd.DataFrame(new_name_dfvec1['pooler_output_avg_actual'].tolist())
XtransformerTest = XvectTest

In [31]:
XtransformerTest

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.353361,-0.243320,0.056097,0.115360,-0.032791,0.398964,0.301549,0.404506,-0.237792,0.364938,...,0.147442,0.174253,0.112246,0.858475,0.286932,0.286815,0.252810,-0.182178,0.195503,0.093186
1,0.325935,-0.239992,0.280767,0.152410,0.015082,0.282491,0.364922,0.154419,-0.194627,0.162829,...,0.064095,0.280048,0.064118,0.568542,0.047745,0.242388,0.178616,0.015276,0.197861,0.091460
2,-0.001449,0.051370,-0.053476,0.482730,0.173749,0.054022,0.036912,0.086766,-0.061818,0.101356,...,-0.012415,-0.131751,0.048086,-0.306986,0.090276,0.104646,-0.025375,0.092447,-0.062682,-0.066849
3,0.208560,-0.370451,0.126028,-0.065593,-0.078183,0.360366,0.338238,0.332010,-0.474838,0.502958,...,0.440072,0.244190,0.343343,0.940398,0.311224,0.537038,0.237087,-0.248849,0.175646,0.350933
4,0.466791,-0.383144,0.377967,-0.367860,-0.222945,0.490231,0.298619,0.459677,-0.317014,0.480612,...,0.397229,0.282514,0.285608,0.973118,0.318171,0.385401,0.282982,-0.269908,0.217813,0.377844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56510,0.332106,-0.392372,0.172117,-0.162470,-0.224149,0.374999,0.383287,0.481330,-0.373353,0.541798,...,0.272140,0.385334,0.295373,0.848848,0.428783,0.255324,0.198023,-0.213244,0.424242,0.236330
56511,0.145826,0.058097,0.017973,0.434786,0.036980,-0.121070,0.264522,0.193853,0.059422,-0.073348,...,-0.190665,0.028466,-0.090524,-0.377234,-0.025598,0.168965,0.086155,0.136396,0.111881,0.034692
56512,0.182500,-0.120713,0.033567,0.150347,0.064818,0.180769,0.222143,0.083916,-0.268578,0.238008,...,0.058859,0.135813,0.255073,0.441894,0.087115,0.203512,0.138748,0.038557,0.123025,0.052619
56513,0.279929,-0.047370,0.088587,0.241009,0.081374,0.135781,0.195505,0.128051,-0.148426,0.159117,...,0.046987,0.083372,-0.014492,0.323009,0.034461,0.197977,0.108623,0.166584,0.027441,0.034798


In [32]:
XtransformerTrain

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,0.291525,-0.210746,0.181804,0.038432,0.006411,0.408258,0.255462,0.270870,-0.228698,0.360879,...,0.094901,0.242647,0.298531,0.887655,0.259375,0.178826,0.290616,-0.189212,0.186709,0.236227
1,0.355184,-0.122825,0.106982,0.087670,-0.064466,0.455416,0.271216,0.301015,-0.364498,0.468364,...,0.360351,0.288066,0.309938,0.922270,0.447318,0.382771,0.321149,-0.275853,0.172854,0.150588
2,0.145432,-0.195878,-0.003989,0.165910,0.059242,0.156935,0.406105,0.101063,-0.150564,0.158669,...,0.206326,0.098941,0.084196,0.737901,0.184666,0.231481,0.159205,-0.019073,-0.026804,0.126507
3,0.197709,-0.216587,0.014209,0.133947,-0.140066,0.321449,0.219419,0.374435,-0.236372,0.427011,...,0.249750,0.211238,0.383206,0.822913,0.301388,0.284200,0.206761,-0.158048,0.316870,0.144445
4,0.275533,-0.254906,0.099941,0.128310,-0.224708,0.429375,0.294607,0.346961,-0.325809,0.537468,...,0.263853,0.336808,0.421352,0.926895,0.382166,0.368388,0.247825,-0.326875,0.402611,0.151677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366,0.244746,-0.108858,0.195402,0.133275,0.000744,0.277244,0.309675,0.113961,-0.256936,0.271037,...,0.145696,0.225685,0.113510,0.660255,0.171794,0.331795,0.240476,0.006964,0.051895,0.095735
1367,0.152325,-0.148160,0.035731,0.296993,-0.054711,0.193614,0.173485,0.067356,-0.053412,0.004537,...,-0.036856,0.159636,0.047876,0.349598,-0.008012,0.283824,0.116508,0.019093,0.092034,0.061417
1368,0.242972,-0.232341,0.127411,0.040343,-0.161518,0.387960,0.270041,0.346398,-0.320948,0.485050,...,0.263049,0.238916,0.368530,0.862730,0.317864,0.273502,0.194048,-0.132383,0.261055,0.124519
1369,0.132832,-0.091226,0.207178,0.251213,0.052481,0.131437,0.125123,0.184050,-0.110585,0.014858,...,0.254450,-0.010142,0.066076,0.400058,0.109091,0.133152,0.280741,0.104474,0.033486,0.005957


In [33]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.svm import SVC
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.multioutput import MultiOutputClassifier
# from sklearn.svm import SVC
# from sklearn.neural_network import MLPClassifier
# from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier, StackingClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier

# estimators = [
#     ('lr', LogisticRegression(max_iter=1000, class_weight='balanced')),
#     ('rf', RandomForestClassifier(n_estimators=100, class_weight='balanced')),
#     # ('gb', GradientBoostingClassifier(n_estimators=100)),
#     # ('xgb', XGBClassifier(n_estimators=100)),
#     ('cat', CatBoostClassifier(iterations=1000, verbose=0)),
#     ('mlp', MLPClassifier(
#     hidden_layer_sizes = (256,512),
#     activation = 'relu',
#     solver = 'adam',
#     alpha = 0.01,
#     learning_rate = 'adaptive',
#     max_iter = 1000,
#     batch_size = 200,
#     momentum = 0.90,
#     learning_rate_init = 0.002))
# ]

# stacking = StackingClassifier(estimators=estimators, final_estimator=MLPClassifier(
#     hidden_layer_sizes = (256,512),
#     activation = 'relu',
#     solver = 'adam',
#     alpha = 0.01,
#     learning_rate = 'adaptive',
#     max_iter = 1000,
#     batch_size = 200,
#     momentum = 0.90,
#     learning_rate_init = 0.002))
# # models.append(('SVM', MultiOutputClassifier(SVC())))
# # models.append(('OvR_AdaBoost', MultiOutputClassifier(AdaBoostClassifier(n_estimators=100))))
# # models.append(('Bagging', MultiOutputClassifier(BaggingClassifier(n_estimators=100))))

In [34]:
# X_train = XtransformerTrain
# X_test = XtransformerTest
# y_train = ytransformerTrain

# # X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# # Обучение модели на тренировочных данных
# multi_target_stacking = MultiOutputClassifier(stacking)

# # Обучение модели на тренировочных данных
# multi_target_stacking.fit(X_train, y_train)

# y_pred = multi_target_stacking.predict(X_test)
# y_pred

In [35]:
# import time

# time.ctime()

In [36]:
# import numpy as np
# from sklearn.neural_network import MLPClassifier
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import classification_report

# X_train = XtransformerTrain
# X_test = XtransformerTest
# y_train = ytransformerTrain

# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


# param_grid = {
#     'hidden_layer_sizes': [(256,512)],
#     'activation': ['relu'],
#     'solver': ['adam'],
#     'alpha': [0.01],
#     'learning_rate': ['adaptive'],
#     'max_iter': [1000],
#     'batch_size': [200],
#     'momentum': [0.90],
#     'learning_rate_init': [0.002],
# }

# mlp = MLPClassifier()

# grid_search = GridSearchCV(estimator=mlp, param_grid=param_grid, n_jobs=-1, cv=3, scoring='f1_samples', verbose=1)

# grid_search.fit(X_train, y_train)

# results = grid_search.cv_results_
# for mean_score, params in zip(results['mean_test_score'], results['params']):
#     print(f"Mean f1_samples score: {mean_score:.4f}")
#     print(f"Parameters: {params}")
#     print()

# print("Лучшие параметры: ", grid_search.best_params_)

# best_model = grid_search.best_estimator_

In [37]:
# # Optional: Evaluate on validation set
# y_pred = best_model.predict(X_val)
# print(classification_report(y_val, y_pred))

In [38]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train = XtransformerTrain
X_test = XtransformerTest
y_train = ytransformerTrain

rf = MultiOutputClassifier(RandomForestClassifier( 
    random_state=42      
))

rf.fit(X_train, y_train)

MultiOutputClassifier(estimator=RandomForestClassifier(random_state=42))

In [39]:
y_pred = rf.predict(X_test)
y_pred

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [40]:
y_pred = pd.DataFrame(y_pred, columns=mlbtransformerTrain.classes_)
y_pred.insert(0, 'iddoc', new_name_dfvec1['iddoc'])
y_pred

,iddoc,Метрология,Охрана и безопасность труда,Охрана окружающей среды,Оценка соответствия,Пожарная безопасность,Промышленная безопасность,Санитарные нормы и правила
0,1400040,0,0,0,0,0,0,0
1,1400051,0,0,0,0,0,0,1
2,5200025,0,0,0,0,0,0,0
3,5200058,0,0,0,0,0,0,0
4,5200086,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...
56510,1306528563,0,0,0,0,0,0,0
56511,1306588785,0,1,0,0,0,0,0
56512,1306610181,0,0,0,0,0,0,0
56513,1306663466,0,0,0,0,0,0,0


In [133]:
y_pred.to_csv('./work_data/RF2nameee_task1_train1371_test56515.csv', index=False)